# Libraries

In [ ]:
import csv
from sklearn.metrics import precision_recall_fscore_support
import seaborn as sns
from scipy.stats import wilcoxon
from collections import OrderedDict
import pickle
import plotly.express as px
from csv import DictWriter
from matplotlib_venn import venn2, venn2_circles
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math
import imblearn
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
import gc
import warnings
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.metrics import matthews_corrcoef
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, TensorDataset
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score
from numpy import asarray,savez_compressed
pd.set_option("display.max_rows", None, "display.max_columns", None)
!pip install -q SentencePiece transformers
from transformers import AdamW, get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup, T5EncoderModel, T5Tokenizer
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.nn.utils.rnn import pad_sequence
import requests
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from sklearn import metrics
#from torch.utils import data
import re
import os
!pip install Biopython

# Functions

In [ ]:
class Classifier2L(nn.Module):
    def __init__(self, hidden, hidden2, dropout=0):
        super(Classifier2L, self).__init__()
        self.hidden = hidden
        self.hidden2 = hidden2
        self.num_feature = 2200
        self.dropout = dropout
        self.batchnorm1 = nn.BatchNorm1d(self.hidden)
        self.batchnorm2 = nn.BatchNorm1d(self.hidden2)

        self.layer_1 = nn.Linear(self.num_feature,  self.hidden)
        self.layer_2 = nn.Linear( self.hidden, self.hidden2)
        self.layer_3 = nn.Linear( self.hidden2, 1)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.layer_1(x)
        x = self.batchnorm1(x)
        x= self.dropout(x)
        x = self.relu(x)

        x = self.layer_2(x)
        x = self.batchnorm2(x)
        x = self.relu(x)
        x= self.dropout(x)

        x = self.layer_3(x)
        #x = self.sigmoid(x)

        return x


    def compute_l1_loss(self, w):
        return torch.abs(w).sum()

    def compute_l2_loss(self, w):
        return torch.square(w).sum()

In [ ]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [ ]:
def notNaN(num):
    return num == num

In [ ]:
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

In [ ]:
# implementation of a single training epoch
def train_epoch_cross(net, loader, loss_fn, optimizer, scheduler):

    # set the network in training mode
    net.train()

    # keep track of the loss
    loss_cum = 0
    cnt = 0
    num_correct = 0
    targets = []
    predictions = []

    for i, data in enumerate(loader):

        # sample data
        x, y = data
        x = x.to(device)
        y = y.to(device)

        # set all gradients equal to zero
        net.zero_grad()

        # feed the batch to the network and compute the outputs
        y_pred = net(x)
        #y_pred_sq = np.squeeze(y_pred) # add when using loss_fn: BCEloss()

        pred = torch.round(y_pred.squeeze())  # rounds to the nearest integer

        # compare predictions to true label
        #correct_tensor = pred.eq(y.float().view_as(pred))
        #correct = np.squeeze(correct_tensor.cpu().numpy())
        #num_correct += np.sum(correct)
        targets.extend(y.cpu().detach().numpy().tolist())
        predictions.extend(torch.sigmoid(y_pred).cpu().detach().numpy().tolist())

        #predictions_round = [round(p) for p in predictions]
        #correct_tensor = predictions_round.eq(targets.float().view_as(predictions_round))

        #loss = loss_fn(y_pred_sq, y.float())  #add when using loss_fn: BCEloss()
        loss = loss_fn(y_pred, y.float())

        # Specify L1 and L2 weights
        l1_weight = 0
        l2_weight = 0

        # Compute L1 and L2 loss component
        parameters = []
        for parameter in net.parameters():
            parameters.append(parameter.view(-1))
        l1 = l1_weight * net.compute_l1_loss(torch.cat(parameters))
        l2 = l2_weight * net.compute_l2_loss(torch.cat(parameters))

        # Add L1 and L2 loss components
        loss += l1
        loss += l2

        loss_cum += loss.data.cpu().numpy()
        cnt += 1

        # backpropagate the gradients w.r.t. computed loss
        loss.backward()

        # apply one step in the optimization
        optimizer.step()
        scheduler.step()

        #break #for training one batch

    # compute the average loss
    #mytrain_acc = num_correct
    loss_avg = loss_cum / cnt

    #predictions = np.array(predictions) >= 0.5
    predictions = [1 if p[0] > 0.5 else 0 for p in predictions]
    mytrain_acc  = metrics.accuracy_score(targets, predictions)   *100
    #mytrain_acc = (num_correct/ (len(y)*cnt)) *100
    return loss_avg, mytrain_acc

def loss_fn(outputs, targets):
  return nn.BCEWithLogitsLoss()(outputs, targets.view(-1, 1))

In [ ]:
# implementation of a single testing epoch
def test_epoch(net, loader, loss_fn):

    # set the network in evaluation mode
    net.eval()

    # keep track of the loss
    loss_cum = 0
    cnt = 0
    num_correct_val = 0
    targets = []
    predictions = []
    with torch.no_grad():
      for i, data in enumerate(loader):

          x, y = data
          x = x.to(device)
          y = y.to(device)

          # feed the batch to the network and compute the outputs
          y_pred = net(x.float())
          #y_pred_sq = np.squeeze(y_pred) #add when using loss_fn: BCEloss()
          pred = torch.round(y_pred.squeeze())  # rounds to the nearest integer

          # compare predictions to true label
          #correct_tensor = pred.eq(y.float().view_as(pred))
          #correct = np.squeeze(correct_tensor.cpu().numpy())

          targets.extend(y.cpu().detach().numpy().tolist())
          predictions.extend(torch.sigmoid(y_pred).cpu().detach().numpy().tolist())

          #num_correct_val += np.sum(correct)
          # compare the outputs to the labels with the loss function
          #loss = loss_fn(y_pred_sq, y.float())  #ad|d when using loss_fn: BCEloss()
          loss = loss_fn(y_pred, y.float())

          # Specify L1 and L2 weights
          l1_weight = 0
          l2_weight = 0

          # Compute L1 and L2 loss component
          parameters = []
          for parameter in net.parameters():
              parameters.append(parameter.view(-1))
          l1 = l1_weight * net.compute_l1_loss(torch.cat(parameters))
          l2 = l2_weight * net.compute_l2_loss(torch.cat(parameters))

          # Add L1 and L2 loss components
          loss += l1
          loss += l2

          loss_cum += loss.data.cpu().numpy()
          cnt += 1

    # compute the average loss
    loss_avg = loss_cum / cnt
    predictions = np.array(predictions) >= 0.5
    mytest_acc  = (metrics.accuracy_score(targets, predictions)) *100
    total = cnt *len(y)

    return loss_avg, mytest_acc

In [ ]:
def train_net_cross(net, train_loader, val_loader, loss_fn, optimizer, epochs , scheduler):

    # transfer the network to the GPU
    net = net.to(device)

    train_loss = np.zeros((epochs))
    test_loss = np.zeros((epochs))
    train_acc = np.zeros((epochs))
    train_acc_alt = np.zeros((epochs))

    test_acc = np.zeros((epochs))
    test_acc_alt = np.zeros((epochs))

    print("Begin training.")

    for epoch in tqdm(range(epochs)):

        # training
        train_loss[epoch], train_acc[epoch] = train_epoch_cross(net, train_loader, loss_fn, optimizer, scheduler)
        # validation
        test_loss[epoch], test_acc[epoch] = test_epoch(net, val_loader, loss_fn)

        print('Epoch %5d - Train loss: %.6f - Train accuracy: %.6f - Test loss: %.6f - Test accuracy: %.6f'
             % (epoch, train_loss[epoch], train_acc[epoch], test_loss[epoch], test_acc[epoch]))

        #print('Epoch %5d - Train loss: %.6f - Train accuracy: %.6f'
        #    % (epoch, train_loss[epoch], train_acc[epoch]))# for training one batch

    return (train_loss, test_loss), (train_acc , test_acc)
    #return train_loss, train_acc # for training one batch

In [ ]:
def dataset_with_indices(cls):
    """
    Modifies the given Dataset class to return a tuple data, target, index
    instead of just data, target.
    """

    def __getitem__(self, index):
        data, target = cls.__getitem__(self, index)
        return data, target, index

    return type(cls.__name__, (cls,), {
        '__getitem__': __getitem__,
    })

DWithInd = dataset_with_indices(torch.utils.data.TensorDataset)

In [ ]:
def Average(lst):
  return sum(lst) /len(lst)

In [ ]:
def get_class_distribution_binary(obj):
    count_dict = {
        'Neutral': 0,
        'Deleterious': 0,
    }

    for i in obj:
        if i == 0:
            count_dict['Neutral'] += 1
        elif i == 1:
            count_dict['Deleterious'] += 1

        else:
            print("Check classes.")

    return count_dict

In [ ]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)

    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)

    acc = torch.round(acc * 100)

    return acc

In [ ]:
class ClassifierDataset(Dataset):

    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__ (self):
        return len(self.X_data)

In [ ]:
def unique(list1):

    # initialize a null list
    unique_list = []

    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)

    print(f'{len(unique_list)} unique transcripts')
    return unique_list

In [ ]:
def init_weights(m):
    if type(m) == nn.Linear:
        #nn.init.normal_(m.weight, std=0.01)
        nn.init.xavier_normal_(m.weight)

    if type(m) == nn.LSTM:
      for param in m._flat_weights_names:
          if "weight" in param:
              nn.init.xavier_uniform_(m._parameters[param])

# Import training set

In [ ]:
# import difference for each mutation + labels
# File found in Datasets found in Zenodo repository
dif_mut = pd.read_csv('log_probWT_MUT_Tier1_2_3_common_balanced+-2_2200AA_57maxpool.csv', header = None, names = ['mutation', 'log_difference', 'label'])

# log_difference has been saved as string of list! Here I convert it back to list of floats
fl_dif = []
for x in dif_mut['log_difference']:
  p = x[1:-1].split(',')
  fl_dif.append([float(i) for i in p])

dif_mut['fl_dif'] = fl_dif

print('Deleterious mutations in dataset: ', len(dif_mut[dif_mut['label'] == 1]))
print('Benign mutations in dataset: ', len(dif_mut[dif_mut['label'] == 0]))

# pad to 2200 AA
N= 2200
fl_dif_pad =[]
for i, mut in dif_mut.iterrows():
  a = mut['fl_dif']
  new_a = a + [0] * (N - len(a))
  fl_dif_pad.append(new_a)
dif_mut['fl_dif_pad'] = fl_dif_pad

#dif_mut_shuffled = dif_mut.sample(frac=1)
dif_mut_shuffled = dif_mut
stacked_flat =[]
for i, mut in dif_mut_shuffled.iterrows():
  stacked_flat.append(torch.tensor(mut['fl_dif_pad']))

stacked_gmm = torch.stack(stacked_flat)
print(stacked_gmm.shape)

# with validation to find early stopping

In [ ]:
temp = dif_mut_shuffled.mutation.str.split(pat='_',expand=True)
# seperate inputs and outputs
X = stacked_gmm
y = dif_mut_shuffled['label']

# Split, shuffle and stratify sequences, mutations, proteins and label
#X_train, X_test, y_train, y_test, mu_all_train, mu_all_test , prot_all_train, prot_all_test = train_test_split(X, y, temp[1],  temp[0], test_size=0.1, stratify=y, random_state=69)
X_train, X_test, y_train, y_test, mu_all_train, mu_all_test , prot_all_train, prot_all_test = train_test_split(X, y, temp[1],  temp[0], test_size=0.0002, stratify=y, random_state=69) #balance_ccommo
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)

#del stacked_gmm, X
gc.collect()

train_dataset = ClassifierDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
val_dataset = ClassifierDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long())

#fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(25,7))# Train
#sns.barplot(data = pd.DataFrame.from_dict([get_class_distribution_binary(y_train)]).melt(), x = "variable", y="value", hue="variable",  ax=axes[0]).set_title('Class Distribution in Train Set')# Validation
#sns.barplot(data = pd.DataFrame.from_dict([get_class_distribution_binary(y_test)]).melt(), x = "variable", y="value", hue="variable",  ax=axes[1]).set_title('Class Distribution in Val Set')

In [ ]:
BATCH_SIZE=64
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, drop_last=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=2, drop_last=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Set fixed random number seed
torch.manual_seed(42)
TRAIN_BATCH_SIZE = 64
epochs =200
lr = 3e-5
#h = 1024
#h = 2048
h = 4096
#h= 8192
#hidden2 =258
#hidden2=1024
hidden2=2048
#hidden2=4096

my_net = Classifier2L(h, hidden2, 0.3).to(device)
#my_net = Classifier(h, 0.3).to(device)
my_net.apply(init_weights)
param_optimizer = list(my_net.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {
        "params": [
            p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.001,
    },
    {
        "params": [
            p for n, p in param_optimizer if any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.02,
    },
]

num_train_steps = int(len(X_train) / TRAIN_BATCH_SIZE * epochs)
optimizer = torch.optim.AdamW(optimizer_parameters, lr=lr)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=1000, num_training_steps=num_train_steps
)
#loss_fn = nn.BCELoss()

loss, acc = train_net_cross(my_net, train_loader, val_loader, loss_fn, optimizer, epochs, scheduler)

plt.rcParams["figure.figsize"] = (15,5)
sns.set()
# plot train/validation loss curves
plt.subplot(1, 2, 1)
plt.plot(loss[0])
plt.plot(loss[1])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title(f'Learning rate: {lr}, hidden : {h}')
plt.legend(('Train', 'Validation'))
#plt.title(f'Learning rate: {lr}, hidden : {h}, hidden2: {h2}')

plt.subplot(1, 2, 2)
plt.plot(acc[0])
plt.plot(acc[1])
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(('Train', 'Validation'))
plt.show()

In [ ]:
# RUN ONCE
#torch.save(my_net.state_dict(), '/content/drive/MyDrive/my_colab/3rdYear/GMM/model')

# Cross-validation DRGN

In [ ]:
# import difference for each mutation + labels
dif_mut = pd.read_csv('/content/drive/MyDrive/log_probWT_MUT_Tier1_2_3_common_balanced+-2_2200AA_57maxpool.csv', header = None, names = ['mutation', 'log_difference', 'label'])

# exclude duplicates
#dif_mut.drop_duplicates(subset = 'mutation', inplace =True)

# log_difference has been saved as string of list! Here I convert it back to list of floats
fl_dif = []
for x in dif_mut['log_difference']:
  p = x[1:-1].split(',')
  fl_dif.append([float(i) for i in p])

dif_mut['fl_dif'] = fl_dif

print('Deleterious mutations in dataset: ', len(dif_mut[dif_mut['label'] == 1]))
print('Benign mutations in dataset: ', len(dif_mut[dif_mut['label'] == 0]))


# pad to 2200 AA
N= 2200
fl_dif_pad =[]
for i, mut in dif_mut.iterrows():
  a = mut['fl_dif']
  new_a = a + [0] * (N - len(a))
  fl_dif_pad.append(new_a)
dif_mut['fl_dif_pad'] = fl_dif_pad
dif_mut_shuffled = dif_mut.sample(frac=1)

In [ ]:
dif_mut_drgn = pd.read_csv('/content/drive/MyDrive/log_probWT_MUT_DRGN_2200AA_57maxpool.csv', header = None, names = ['mutation', 'log_difference', 'label'])
# exclude duplicates
#dif_mut_drgn.drop_duplicates(subset = 'mutation', inplace =True)

# log_difference has been saved as string of list! Here I convert it back to list of floats
fl_dif = []
for x in dif_mut_drgn['log_difference']:
  p = x[1:-1].split(',')
  fl_dif.append([float(i) for i in p])

dif_mut_drgn['fl_dif'] = fl_dif

print('Deleterious mutations in dataset: ', len(dif_mut_drgn[dif_mut_drgn['label'] == 1]))
print('Benign mutations in dataset: ', len(dif_mut_drgn[dif_mut_drgn['label'] == 0]))

# pad to 2200 AA
N= 2200
fl_dif_pad =[]
for i, mut in dif_mut_drgn.iterrows():
  a = mut['fl_dif']
  new_a = a + [0] * (N - len(a))
  fl_dif_pad.append(new_a)
dif_mut_drgn['fl_dif_pad'] = fl_dif_pad

stacked_flat_drgn =[]
for i, mut in dif_mut_drgn.iterrows():
  stacked_flat_drgn.append(torch.tensor(mut['fl_dif_pad']))

dif_mut_drgn_shuffled = dif_mut_drgn.sample(frac=1)

In [ ]:
# implementation of a single training epoch
def train_epoch_cross(net, loader, loss_fn, optimizer, scheduler):

    # set the network in training mode
    net.train()

    # keep track of the loss
    loss_cum = 0
    cnt = 0
    num_correct = 0
    targets = []
    predictions = []

    for i, data in enumerate(loader):

        # sample data
        x, y = data
        x = x.to(device)
        y = y.to(device)

        # set all gradients equal to zero
        net.zero_grad()

        # feed the batch to the network and compute the outputs
        y_pred = net(x)
        #y_pred_sq = np.squeeze(y_pred) # add when using loss_fn: BCEloss()

        pred = torch.round(y_pred.squeeze())  # rounds to the nearest integer

        # compare predictions to true label
        #correct_tensor = pred.eq(y.float().view_as(pred))
        #correct = np.squeeze(correct_tensor.cpu().numpy())
        #num_correct += np.sum(correct)
        targets.extend(y.cpu().detach().numpy().tolist())
        predictions.extend(torch.sigmoid(y_pred).cpu().detach().numpy().tolist())

        #predictions_round = [round(p) for p in predictions]
        #correct_tensor = predictions_round.eq(targets.float().view_as(predictions_round))

        #loss = loss_fn(y_pred_sq, y.float())  #add when using loss_fn: BCEloss()
        loss = loss_fn(y_pred, y.float())

        # Specify L1 and L2 weights
        l1_weight = 0
        l2_weight = 0

        # Compute L1 and L2 loss component
        parameters = []
        for parameter in net.parameters():
            parameters.append(parameter.view(-1))
        l1 = l1_weight * net.compute_l1_loss(torch.cat(parameters))
        l2 = l2_weight * net.compute_l2_loss(torch.cat(parameters))

        # Add L1 and L2 loss components
        loss += l1
        loss += l2

        loss_cum += loss.data.cpu().numpy()
        cnt += 1

        # backpropagate the gradients w.r.t. computed loss
        loss.backward()

        # apply one step in the optimization
        optimizer.step()
        scheduler.step()

        #break #for training one batch

    # compute the average loss
    #mytrain_acc = num_correct
    loss_avg = loss_cum / cnt

    #predictions = np.array(predictions) >= 0.5
    predictions = [1 if p[0] > 0.5 else 0 for p in predictions]
    mytrain_acc  = metrics.accuracy_score(targets, predictions)   *100
    #mytrain_acc = (num_correct/ (len(y)*cnt)) *100
    return loss_avg, mytrain_acc

def loss_fn(outputs, targets):
  return nn.BCEWithLogitsLoss()(outputs, targets.view(-1, 1))

In [ ]:
def train_net_cross(net, train_loader, val_loader, loss_fn, optimizer, epochs , scheduler):

    # transfer the network to the GPU
    net = net.to(device)

    train_loss = np.zeros((epochs))
    test_loss = np.zeros((epochs))
    train_acc = np.zeros((epochs))
    train_acc_alt = np.zeros((epochs))

    test_acc = np.zeros((epochs))
    test_acc_alt = np.zeros((epochs))

    print("Begin training.")

    for epoch in tqdm(range(epochs)):

        # training
        train_loss[epoch], train_acc[epoch] = train_epoch_cross(net, train_loader, loss_fn, optimizer, scheduler)
        #predictions, outputs = eval(net, train_loader)
        #train_acc_alt[epoch] = metrics.accuracy_score(outputs, predictions)

        # validation
        test_loss[epoch], test_acc[epoch] = test_epoch(net, val_loader, loss_fn)
        #predictions, outputs = eval(net, val_loader)
        #test_acc_alt[epoch] = metrics.accuracy_score(outputs, predictions)

        #print('Epoch %5d - Train loss: %.6f - Train accuracy: %.6f - Test loss: %.6f - Test accuracy: %.6f'
        #     % (epoch, train_loss[epoch], train_acc[epoch], test_loss[epoch], test_acc[epoch]))

        #print('Epoch %5d - Train loss: %.6f - Train accuracy: %.6f'
        #    % (epoch, train_loss[epoch], train_acc[epoch]))# for training one batch

    return (train_loss, test_loss), (train_acc , test_acc)
    #return train_loss, train_acc # for training one batch

In [ ]:
# Distribution of proteins in trianing / test se tof fold
# Set fixed random number seed
torch.manual_seed(42)
TRAIN_BATCH_SIZE = 64
epochs=200
lr = [3e-5]
#h = 1024
#h = 2048
h = 4096
#hidden2 =258
#hidden2=1024
hidden2=2048
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

n_folds = 5

accuracy, sensitivity, specificity, mcc, auc_dict = {}, {}, {}, {}, {}

for fold in range(n_folds):
  test = dif_mut_drgn_shuffled[fold*round(len(dif_mut_drgn_shuffled)/n_folds):(fold+1)*round(len(dif_mut_drgn_shuffled)/n_folds)]
  temp = test.mutation.str.split(pat='_',expand=True)[0]


  stacked_flat_drgn =[]
  for i, mut in test.iterrows():
    stacked_flat_drgn.append(torch.tensor(mut['fl_dif_pad']))

  stacked_drgn = torch.stack(stacked_flat_drgn)
  print(stacked_drgn.shape)

  cond = dif_mut_shuffled['mutation'].isin(test['mutation'])
  training = dif_mut_shuffled.drop(dif_mut_shuffled[cond].index, inplace = False)

  #print('deleterious labels in Training for Fold ', fold, ': ', len(training[training['label']==1]))
  #print('benign labels in Training for Fold ', fold, ': ',  len(training[training['label']==0]))

  #print('deleterious labels in Test for Fold ', fold, ': ', len(test[test['label']==1]))
  #print('benign labels in Test for Fold ', fold, ': ',  len(test[test['label']==0]))
  #print('different genes in Fold ', fold, ': ',  len(temp.unique()))


  # exclude test mutations from training data
  stacked_flat =[]
  for i, mut in training.iterrows():
    stacked_flat.append(torch.tensor(mut['fl_dif_pad']))

  stacked_gmm = torch.stack(stacked_flat)
  print(stacked_gmm.shape)

  training_temp = training.mutation.str.split(pat='_',expand=True)

  # how many proteins co exist in training set / test set
  prot_inters= intersection(training_temp[0].unique(), temp.unique())
  print('Proteins in common between training set / test set: ', len(prot_inters))#, ': ', prot_inters)
  '''
  # how many deleterious / benign per protein are in training versus test set
  for prot_int in prot_inters:
    training_df = training[training['mutation'].str.contains(prot_int)]
    del_training = len(training_df[training_df['label'] == 1])
    ben_training = len(training_df[training_df['label'] == 0])

    test_df =  test[test['mutation'].str.contains(prot_int)]
    del_test = len(test_df[test_df['label'] == 1])
    ben_test = len(test_df[test_df['label'] == 0])

    # creating the dataset
    data = {'Train del':del_training, 'Train ben':ben_training, 'Test del':del_test, 'Test ben':ben_test}
    courses = list(data.keys())
    values = list(data.values())

    fig = plt.figure(figsize = (5, 3))

    # creating the bar plot
    plt.bar(courses, values, color ='maroon',
            width = 0.4)

    plt.xlabel("")
    plt.ylabel("No. of samples")
    plt.title(prot_int)
    plt.xticks(rotation=45)
    plt.show()

  '''
  # seperate inputs and outputs
  X = stacked_gmm
  y = training['label']

  # Split, shuffle and stratify sequences, mutations, proteins and label
  X_train, X_test, y_train, y_test, mu_all_train, mu_all_test , prot_all_train, prot_all_test = train_test_split(X, y, training_temp[1],  training_temp[0], test_size=0.0002, stratify=y, random_state=69)
  X_train, y_train = np.array(X_train), np.array(y_train)
  X_test, y_test = np.array(X_test), np.array(y_test)

  del stacked_gmm, X
  gc.collect()

  train_dataset = ClassifierDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
  val_dataset = ClassifierDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long())

  #fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(25,7))# Train
  #sns.barplot(data = pd.DataFrame.from_dict([get_class_distribution_binary(y_train)]).melt(), x = "variable", y="value", hue="variable",  ax=axes[0]).set_title('Class Distribution in Train Set')# Validation
  #sns.barplot(data = pd.DataFrame.from_dict([get_class_distribution_binary(y_test)]).melt(), x = "variable", y="value", hue="variable",  ax=axes[1]).set_title('Class Distribution in Val Set')

  train_loader = DataLoader(dataset=train_dataset, batch_size=TRAIN_BATCH_SIZE, drop_last=True)
  val_loader = DataLoader(dataset=val_dataset, batch_size=2, drop_last=True)

  my_net = Classifier2L(h, hidden2, 0.3).to(device)
  #my_net = my_net = Classifier(h, 0.3).to(device)
  my_net.apply(init_weights)
  param_optimizer = list(my_net.named_parameters())
  no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
  optimizer_parameters = [
      {
          "params": [
              p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
          ],
          "weight_decay": 0.001,
      },
      {
          "params": [
              p for n, p in param_optimizer if any(nd in n for nd in no_decay)
          ],
          "weight_decay": 0.02,
      },
  ]

  num_train_steps = int(len(X_train) / TRAIN_BATCH_SIZE * epochs)
  optimizer = torch.optim.AdamW(optimizer_parameters, lr=3e-5)
  scheduler = get_linear_schedule_with_warmup(
      optimizer, num_warmup_steps=1000, num_training_steps=num_train_steps
  )

  loss, acc = train_net_cross(my_net, train_loader, val_loader, loss_fn, optimizer, epochs, scheduler )


  labels_drgn = test['label'].tolist()
  X_drgn, y_drgn = np.array(stacked_drgn), np.array(labels_drgn)

  drgn_dataset = ClassifierDataset(torch.from_numpy(X_drgn).float(), torch.from_numpy(y_drgn).long())
  drgn_loader = DataLoader(dataset=drgn_dataset, batch_size=1 , drop_last=True)

  y_pred_list = []
  predictions_drgn= []
  my_net.eval()

  with torch.no_grad():
      for X_batch, _ in drgn_loader:
          X_batch = X_batch.to(device)
          y_test_pred = my_net(X_batch)
          #print(y_test_pred)
          predictions_drgn.extend(torch.sigmoid(y_test_pred).cpu().detach().numpy().tolist())
          #print(predictions_drgn)

  flat_list = []
  for sublist in predictions_drgn:
      for item in sublist:
          flat_list.append(item)

  predictions_drgn = [1 if i>=0.5 else 0 for i in flat_list]
  confusion_matrix_df = pd.DataFrame(confusion_matrix(y_drgn, predictions_drgn))
  tn, fp, fn, tp = confusion_matrix(y_drgn, predictions_drgn).ravel()
  print(classification_report(y_drgn, predictions_drgn))
  print(tp, tn, fp, fn)
  TPR =tp / (tp + fn)
  print('Sensitivity: %.2f'  % (TPR*100))

  TNR = tn / (tn + fp)
  print('Specificity: %.2f'  % (TNR*100))
  print(matthews_corrcoef(y_drgn, predictions_drgn))

  lw = 2
  import matplotlib
  matplotlib.rc_file_defaults()
  fpr, tpr, _ = metrics.roc_curve(y_drgn[:len(predictions_drgn)],  flat_list)
  auc = metrics.roc_auc_score(y_drgn[:len(predictions_drgn)], flat_list)
  plt.plot(fpr,tpr,color="navy",label="DRGN, auc="+str(round(auc, 3)))
  plt.plot([0, 1], [0, 1], color="darkorange", lw=lw, linestyle="--")
  plt.legend(loc=4)
  plt.xlabel("False Positive Rate")
  plt.ylabel("True Positive Rate")
  plt.title("Receiver operating characteristic DRGN set")
  plt.rcParams["figure.figsize"] = (10,10)
  plt.show()

  accuracy[fold]=((tp+tn)/(tp +tn+ fp+ fn))*100
  sensitivity[fold]=TPR*100
  specificity[fold]=TNR*100
  mcc[fold]= matthews_corrcoef(y_drgn, predictions_drgn)*100
  auc_dict[fold] =round(auc, 3)*100

  # find the accuracy per protein
  test['Groundtruth'] = y_drgn
  test['prediction'] = predictions_drgn

  training_df = pd.DataFrame(list(zip(prot_all_train.tolist(), y_train.tolist())),
               columns =['uniprot', 'label'])
  for prot_int in prot_inters:

    print('----------')

    df = training_df[training_df['uniprot']==prot_int]
    del_training = len(df[df['label'] == 1])
    ben_training = len(df[df['label'] == 0])
    print(f'Protein {prot_int} has {del_training} deleterious and {ben_training} benign samples in training set of {fold} Fold ')
    test_accu = test[test['mutation'].str.contains(prot_int)]
    del_test_accu = len(test_accu[test_accu['label'] == 1])
    ben_test_accu = len(test_accu[test_accu['label'] == 0])
    print(f'Protein {prot_int} has {del_test_accu} deleterious and {ben_test_accu} benign samples in test set of {fold} Fold ')

    print('Samples for Protein: ', len(test_accu))
    #print(classification_report(test_accu['Groundtruth'], test_accu['prediction']))
    precision, recall, fscore, support = precision_recall_fscore_support(test_accu['Groundtruth'], test_accu['prediction'], average='micro')#, pos_label=1)
    print('Precision: {}'.format(precision))
    print('Recall: {}'.format(recall))
    print('F-measure: {}'.format(fscore))

In [ ]:
ds=[accuracy,  sensitivity,  specificity,  mcc, auc_dict]
test = pd.DataFrame(columns = ['metric', 'fold', 'value'])
fold_list, value_list = [],[]
for i, m in enumerate(ds):
  for key, value in m.items():
    fold_list.append(key)
    value_list.append(value)

test['metric'] = ['accuracy']* len(accuracy) + ['sensitivity']* len(accuracy)+  ['specificity']* len(accuracy)+  ['mcc']* len(accuracy)+ ['AUC']* len(accuracy)
test['fold'] = fold_list
test['value'] = value_list

fig = px.violin(test, y="metric", x="value", color="metric", box=True, points="all", width=800, height=400, title="10 folds")
fig.show()

In [ ]:
sns.set(style="darkgrid")
sns.violinplot(data=test, x="value", y="metric", palette="Blues")
plt.show()

In [ ]:
# Distribution of proteins in trianing / test se tof fold
# Set fixed random number seed
torch.manual_seed(42)
TRAIN_BATCH_SIZE = 64
epochs=200
lr = [3e-5]
#h = 1024
#h = 2048
h = 4096
#hidden2 =258
#hidden2=1024
hidden2=2048
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

n_folds = 10

accuracy, sensitivity, specificity, mcc, auc_dict = {}, {}, {}, {}, {}

for fold in range(n_folds):
  test = dif_mut_drgn_shuffled[fold*round(len(dif_mut_drgn_shuffled)/n_folds):(fold+1)*round(len(dif_mut_drgn_shuffled)/n_folds)]
  temp = test.mutation.str.split(pat='_',expand=True)[0]


  stacked_flat_drgn =[]
  for i, mut in test.iterrows():
    stacked_flat_drgn.append(torch.tensor(mut['fl_dif_pad']))

  stacked_drgn = torch.stack(stacked_flat_drgn)
  print(stacked_drgn.shape)

  cond = dif_mut_shuffled['mutation'].isin(test['mutation'])
  training = dif_mut_shuffled.drop(dif_mut_shuffled[cond].index, inplace = False)

  #print('deleterious labels in Training for Fold ', fold, ': ', len(training[training['label']==1]))
  #print('benign labels in Training for Fold ', fold, ': ',  len(training[training['label']==0]))

  #print('deleterious labels in Test for Fold ', fold, ': ', len(test[test['label']==1]))
  #print('benign labels in Test for Fold ', fold, ': ',  len(test[test['label']==0]))
  #print('different genes in Fold ', fold, ': ',  len(temp.unique()))


  # exclude test mutations from training data
  stacked_flat =[]
  for i, mut in training.iterrows():
    stacked_flat.append(torch.tensor(mut['fl_dif_pad']))

  stacked_gmm = torch.stack(stacked_flat)
  print(stacked_gmm.shape)

  training_temp = training.mutation.str.split(pat='_',expand=True)

  # how many proteins co exist in training set / test set
  prot_inters= intersection(training_temp[0].unique(), temp.unique())
  print('Proteins in common between training set / test set: ', len(prot_inters))#, ': ', prot_inters)
  '''
  # how many deleterious / benign per protein are in training versus test set
  for prot_int in prot_inters:
    training_df = training[training['mutation'].str.contains(prot_int)]
    del_training = len(training_df[training_df['label'] == 1])
    ben_training = len(training_df[training_df['label'] == 0])

    test_df =  test[test['mutation'].str.contains(prot_int)]
    del_test = len(test_df[test_df['label'] == 1])
    ben_test = len(test_df[test_df['label'] == 0])

    # creating the dataset
    data = {'Train del':del_training, 'Train ben':ben_training, 'Test del':del_test, 'Test ben':ben_test}
    courses = list(data.keys())
    values = list(data.values())

    fig = plt.figure(figsize = (5, 3))

    # creating the bar plot
    plt.bar(courses, values, color ='maroon',
            width = 0.4)

    plt.xlabel("")
    plt.ylabel("No. of samples")
    plt.title(prot_int)
    plt.xticks(rotation=45)
    plt.show()

  '''
  # seperate inputs and outputs
  X = stacked_gmm
  y = training['label']

  # Split, shuffle and stratify sequences, mutations, proteins and label
  X_train, X_test, y_train, y_test, mu_all_train, mu_all_test , prot_all_train, prot_all_test = train_test_split(X, y, training_temp[1],  training_temp[0], test_size=0.0002, stratify=y, random_state=69)
  X_train, y_train = np.array(X_train), np.array(y_train)
  X_test, y_test = np.array(X_test), np.array(y_test)

  del stacked_gmm, X
  gc.collect()

  train_dataset = ClassifierDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
  val_dataset = ClassifierDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long())

  #fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(25,7))# Train
  #sns.barplot(data = pd.DataFrame.from_dict([get_class_distribution_binary(y_train)]).melt(), x = "variable", y="value", hue="variable",  ax=axes[0]).set_title('Class Distribution in Train Set')# Validation
  #sns.barplot(data = pd.DataFrame.from_dict([get_class_distribution_binary(y_test)]).melt(), x = "variable", y="value", hue="variable",  ax=axes[1]).set_title('Class Distribution in Val Set')

  train_loader = DataLoader(dataset=train_dataset, batch_size=TRAIN_BATCH_SIZE, drop_last=True)
  val_loader = DataLoader(dataset=val_dataset, batch_size=2, drop_last=True)

  my_net = Classifier2L(h, hidden2, 0.3).to(device)
  #my_net = my_net = Classifier(h, 0.3).to(device)
  my_net.apply(init_weights)
  param_optimizer = list(my_net.named_parameters())
  no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
  optimizer_parameters = [
      {
          "params": [
              p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
          ],
          "weight_decay": 0.001,
      },
      {
          "params": [
              p for n, p in param_optimizer if any(nd in n for nd in no_decay)
          ],
          "weight_decay": 0.02,
      },
  ]

  num_train_steps = int(len(X_train) / TRAIN_BATCH_SIZE * epochs)
  optimizer = torch.optim.AdamW(optimizer_parameters, lr=3e-5)
  scheduler = get_linear_schedule_with_warmup(
      optimizer, num_warmup_steps=1000, num_training_steps=num_train_steps
  )

  loss, acc = train_net_cross(my_net, train_loader, val_loader, loss_fn, optimizer, epochs, scheduler )


  labels_drgn = test['label'].tolist()
  X_drgn, y_drgn = np.array(stacked_drgn), np.array(labels_drgn)

  drgn_dataset = ClassifierDataset(torch.from_numpy(X_drgn).float(), torch.from_numpy(y_drgn).long())
  drgn_loader = DataLoader(dataset=drgn_dataset, batch_size=1 , drop_last=True)


  y_pred_list = []
  predictions_drgn= []
  my_net.eval()

  with torch.no_grad():
      for X_batch, _ in drgn_loader:
          X_batch = X_batch.to(device)
          y_test_pred = my_net(X_batch)
          #print(y_test_pred)
          predictions_drgn.extend(torch.sigmoid(y_test_pred).cpu().detach().numpy().tolist())
          #print(predictions_drgn)

  flat_list = []
  for sublist in predictions_drgn:
      for item in sublist:
          flat_list.append(item)

  predictions_drgn = [1 if i>=0.5 else 0 for i in flat_list]
  confusion_matrix_df = pd.DataFrame(confusion_matrix(y_drgn, predictions_drgn))
  tn, fp, fn, tp = confusion_matrix(y_drgn, predictions_drgn).ravel()
  print(classification_report(y_drgn, predictions_drgn))
  print(tp, tn, fp, fn)
  TPR =tp / (tp + fn)
  print('Sensitivity: %.2f'  % (TPR*100))

  TNR = tn / (tn + fp)
  print('Specificity: %.2f'  % (TNR*100))
  print(matthews_corrcoef(y_drgn, predictions_drgn))

  lw = 2
  import matplotlib
  matplotlib.rc_file_defaults()
  fpr, tpr, _ = metrics.roc_curve(y_drgn[:len(predictions_drgn)],  flat_list)
  auc = metrics.roc_auc_score(y_drgn[:len(predictions_drgn)], flat_list)
  plt.plot(fpr,tpr,color="navy",label="DRGN, auc="+str(round(auc, 3)))
  plt.plot([0, 1], [0, 1], color="darkorange", lw=lw, linestyle="--")
  plt.legend(loc=4)
  plt.xlabel("False Positive Rate")
  plt.ylabel("True Positive Rate")
  plt.title("Receiver operating characteristic DRGN set")
  plt.rcParams["figure.figsize"] = (10,10)
  plt.show()

  accuracy[fold]=((tp+tn)/(tp +tn+ fp+ fn))*100
  sensitivity[fold]=TPR*100
  specificity[fold]=TNR*100
  mcc[fold]= matthews_corrcoef(y_drgn, predictions_drgn)*100
  auc_dict[fold] =round(auc, 3)*100

  # find the accuracy per protein
  test['Groundtruth'] = y_drgn
  test['prediction'] = predictions_drgn

  training_df = pd.DataFrame(list(zip(prot_all_train.tolist(), y_train.tolist())),
               columns =['uniprot', 'label'])
  for prot_int in prot_inters:

    print('----------')

    df = training_df[training_df['uniprot']==prot_int]
    del_training = len(df[df['label'] == 1])
    ben_training = len(df[df['label'] == 0])
    print(f'Protein {prot_int} has {del_training} deleterious and {ben_training} benign samples in training set of {fold} Fold ')
    test_accu = test[test['mutation'].str.contains(prot_int)]
    del_test_accu = len(test_accu[test_accu['label'] == 1])
    ben_test_accu = len(test_accu[test_accu['label'] == 0])
    print(f'Protein {prot_int} has {del_test_accu} deleterious and {ben_test_accu} benign samples in test set of {fold} Fold ')

    print('Samples for Protein: ', len(test_accu))
    #print(classification_report(test_accu['Groundtruth'], test_accu['prediction']))
    precision, recall, fscore, support = precision_recall_fscore_support(test_accu['Groundtruth'], test_accu['prediction'], average='micro')#, pos_label=1)
    print('Precision: {}'.format(precision))
    print('Recall: {}'.format(recall))
    print('F-measure: {}'.format(fscore))

In [ ]:
ds=[accuracy,  sensitivity,  specificity,  mcc, auc_dict]
test = pd.DataFrame(columns = ['metric', 'fold', 'value'])
fold_list, value_list = [],[]
for i, m in enumerate(ds):
  for key, value in m.items():
    fold_list.append(key)
    value_list.append(value)

test['metric'] = ['accuracy']* len(accuracy) + ['sensitivity']* len(accuracy)+  ['specificity']* len(accuracy)+  ['mcc']* len(accuracy)+ ['AUC']* len(accuracy)
test['fold'] = fold_list
test['value'] = value_list

fig = px.violin(test, y="metric", x="value", color="metric", box=True, points="all", width=800, height=400, title="10 folds")
fig.show()

In [ ]:
sns.set(style="darkgrid")
sns.violinplot(data=test, x="value", y="metric", palette="Blues")
plt.show()

In [ ]:
# Distribution of proteins in trianing / test se tof fold
# Set fixed random number seed
torch.manual_seed(42)
TRAIN_BATCH_SIZE = 64
epochs=200
lr = [3e-5]
#h = 1024
#h = 2048
h = 4096
#hidden2 =258
#hidden2=1024
hidden2=2048
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

n_folds = 20

accuracy, sensitivity, specificity, mcc, auc_dict = {}, {}, {}, {}, {}

for fold in range(n_folds):
  test = dif_mut_drgn_shuffled[fold*round(len(dif_mut_drgn_shuffled)/n_folds):(fold+1)*round(len(dif_mut_drgn_shuffled)/n_folds)]
  temp = test.mutation.str.split(pat='_',expand=True)[0]


  stacked_flat_drgn =[]
  for i, mut in test.iterrows():
    stacked_flat_drgn.append(torch.tensor(mut['fl_dif_pad']))

  stacked_drgn = torch.stack(stacked_flat_drgn)
  print(stacked_drgn.shape)

  cond = dif_mut_shuffled['mutation'].isin(test['mutation'])
  training = dif_mut_shuffled.drop(dif_mut_shuffled[cond].index, inplace = False)

  #print('deleterious labels in Training for Fold ', fold, ': ', len(training[training['label']==1]))
  #print('benign labels in Training for Fold ', fold, ': ',  len(training[training['label']==0]))

  #print('deleterious labels in Test for Fold ', fold, ': ', len(test[test['label']==1]))
  #print('benign labels in Test for Fold ', fold, ': ',  len(test[test['label']==0]))
  #print('different genes in Fold ', fold, ': ',  len(temp.unique()))


  # exclude test mutations from training data
  stacked_flat =[]
  for i, mut in training.iterrows():
    stacked_flat.append(torch.tensor(mut['fl_dif_pad']))

  stacked_gmm = torch.stack(stacked_flat)
  print(stacked_gmm.shape)

  training_temp = training.mutation.str.split(pat='_',expand=True)

  # how many proteins co exist in training set / test set
  prot_inters= intersection(training_temp[0].unique(), temp.unique())
  print('Proteins in common between training set / test set: ', len(prot_inters))#, ': ', prot_inters)
  '''
  # how many deleterious / benign per protein are in training versus test set
  for prot_int in prot_inters:
    training_df = training[training['mutation'].str.contains(prot_int)]
    del_training = len(training_df[training_df['label'] == 1])
    ben_training = len(training_df[training_df['label'] == 0])

    test_df =  test[test['mutation'].str.contains(prot_int)]
    del_test = len(test_df[test_df['label'] == 1])
    ben_test = len(test_df[test_df['label'] == 0])

    # creating the dataset
    data = {'Train del':del_training, 'Train ben':ben_training, 'Test del':del_test, 'Test ben':ben_test}
    courses = list(data.keys())
    values = list(data.values())

    fig = plt.figure(figsize = (5, 3))

    # creating the bar plot
    plt.bar(courses, values, color ='maroon',
            width = 0.4)

    plt.xlabel("")
    plt.ylabel("No. of samples")
    plt.title(prot_int)
    plt.xticks(rotation=45)
    plt.show()

  '''
  # seperate inputs and outputs
  X = stacked_gmm
  y = training['label']

  # Split, shuffle and stratify sequences, mutations, proteins and label
  X_train, X_test, y_train, y_test, mu_all_train, mu_all_test , prot_all_train, prot_all_test = train_test_split(X, y, training_temp[1],  training_temp[0], test_size=0.0002, stratify=y, random_state=69)
  X_train, y_train = np.array(X_train), np.array(y_train)
  X_test, y_test = np.array(X_test), np.array(y_test)

  del stacked_gmm, X
  gc.collect()

  train_dataset = ClassifierDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
  val_dataset = ClassifierDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long())

  #fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(25,7))# Train
  #sns.barplot(data = pd.DataFrame.from_dict([get_class_distribution_binary(y_train)]).melt(), x = "variable", y="value", hue="variable",  ax=axes[0]).set_title('Class Distribution in Train Set')# Validation
  #sns.barplot(data = pd.DataFrame.from_dict([get_class_distribution_binary(y_test)]).melt(), x = "variable", y="value", hue="variable",  ax=axes[1]).set_title('Class Distribution in Val Set')

  train_loader = DataLoader(dataset=train_dataset, batch_size=TRAIN_BATCH_SIZE, drop_last=True)
  val_loader = DataLoader(dataset=val_dataset, batch_size=2, drop_last=True)

  my_net = Classifier2L(h, hidden2, 0.3).to(device)
  #my_net = my_net = Classifier(h, 0.3).to(device)
  my_net.apply(init_weights)
  param_optimizer = list(my_net.named_parameters())
  no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
  optimizer_parameters = [
      {
          "params": [
              p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
          ],
          "weight_decay": 0.001,
      },
      {
          "params": [
              p for n, p in param_optimizer if any(nd in n for nd in no_decay)
          ],
          "weight_decay": 0.02,
      },
  ]

  num_train_steps = int(len(X_train) / TRAIN_BATCH_SIZE * epochs)
  optimizer = torch.optim.AdamW(optimizer_parameters, lr=3e-5)
  scheduler = get_linear_schedule_with_warmup(
      optimizer, num_warmup_steps=1000, num_training_steps=num_train_steps
  )

  loss, acc = train_net_cross(my_net, train_loader, val_loader, loss_fn, optimizer, epochs, scheduler )


  labels_drgn = test['label'].tolist()
  X_drgn, y_drgn = np.array(stacked_drgn), np.array(labels_drgn)

  drgn_dataset = ClassifierDataset(torch.from_numpy(X_drgn).float(), torch.from_numpy(y_drgn).long())
  drgn_loader = DataLoader(dataset=drgn_dataset, batch_size=1 , drop_last=True)


  y_pred_list = []
  predictions_drgn= []
  my_net.eval()

  with torch.no_grad():
      for X_batch, _ in drgn_loader:
          X_batch = X_batch.to(device)
          y_test_pred = my_net(X_batch)
          #print(y_test_pred)
          predictions_drgn.extend(torch.sigmoid(y_test_pred).cpu().detach().numpy().tolist())
          #print(predictions_drgn)

  flat_list = []
  for sublist in predictions_drgn:
      for item in sublist:
          flat_list.append(item)

  predictions_drgn = [1 if i>=0.5 else 0 for i in flat_list]
  confusion_matrix_df = pd.DataFrame(confusion_matrix(y_drgn, predictions_drgn))
  tn, fp, fn, tp = confusion_matrix(y_drgn, predictions_drgn).ravel()
  print(classification_report(y_drgn, predictions_drgn))
  print(tp, tn, fp, fn)
  TPR =tp / (tp + fn)
  print('Sensitivity: %.2f'  % (TPR*100))

  TNR = tn / (tn + fp)
  print('Specificity: %.2f'  % (TNR*100))
  print(matthews_corrcoef(y_drgn, predictions_drgn))

  lw = 2
  import matplotlib
  matplotlib.rc_file_defaults()
  fpr, tpr, _ = metrics.roc_curve(y_drgn[:len(predictions_drgn)],  flat_list)
  auc = metrics.roc_auc_score(y_drgn[:len(predictions_drgn)], flat_list)
  plt.plot(fpr,tpr,color="navy",label="DRGN, auc="+str(round(auc, 3)))
  plt.plot([0, 1], [0, 1], color="darkorange", lw=lw, linestyle="--")
  plt.legend(loc=4)
  plt.xlabel("False Positive Rate")
  plt.ylabel("True Positive Rate")
  plt.title("Receiver operating characteristic DRGN set")
  plt.rcParams["figure.figsize"] = (10,10)
  plt.show()

  accuracy[fold]=((tp+tn)/(tp +tn+ fp+ fn))*100
  sensitivity[fold]=TPR*100
  specificity[fold]=TNR*100
  mcc[fold]= matthews_corrcoef(y_drgn, predictions_drgn)*100
  auc_dict[fold] =round(auc, 3)*100

  # find the accuracy per protein
  test['Groundtruth'] = y_drgn
  test['prediction'] = predictions_drgn

  training_df = pd.DataFrame(list(zip(prot_all_train.tolist(), y_train.tolist())),
               columns =['uniprot', 'label'])
  for prot_int in prot_inters:

    print('----------')

    df = training_df[training_df['uniprot']==prot_int]
    del_training = len(df[df['label'] == 1])
    ben_training = len(df[df['label'] == 0])
    print(f'Protein {prot_int} has {del_training} deleterious and {ben_training} benign samples in training set of {fold} Fold ')
    test_accu = test[test['mutation'].str.contains(prot_int)]
    del_test_accu = len(test_accu[test_accu['label'] == 1])
    ben_test_accu = len(test_accu[test_accu['label'] == 0])
    print(f'Protein {prot_int} has {del_test_accu} deleterious and {ben_test_accu} benign samples in test set of {fold} Fold ')

    print('Samples for Protein: ', len(test_accu))
    #print(classification_report(test_accu['Groundtruth'], test_accu['prediction']))
    precision, recall, fscore, support = precision_recall_fscore_support(test_accu['Groundtruth'], test_accu['prediction'], average='micro')#, pos_label=1)
    print('Precision: {}'.format(precision))
    print('Recall: {}'.format(recall))
    print('F-measure: {}'.format(fscore))

In [ ]:
ds=[accuracy,  sensitivity,  specificity,  mcc, auc_dict]
test = pd.DataFrame(columns = ['metric', 'fold', 'value'])
fold_list, value_list = [],[]
for i, m in enumerate(ds):
  for key, value in m.items():
    fold_list.append(key)
    value_list.append(value)

test['metric'] = ['accuracy']* len(accuracy) + ['sensitivity']* len(accuracy)+  ['specificity']* len(accuracy)+  ['mcc']* len(accuracy)+ ['AUC']* len(accuracy)
test['fold'] = fold_list
test['value'] = value_list

fig = px.violin(test, y="metric", x="value", color="metric", box=True, points="all", width=800, height=400, title="20 folds")
fig.show()

In [ ]:
sns.set(style="darkgrid")
sns.violinplot(data=test, x="value", y="metric", palette="Blues")
plt.show()

# Cross-validation DRGN with results of all predictors

In [ ]:
# implementation of a single training epoch
def train_epoch_cross(net, loader, loss_fn, optimizer, scheduler):

    # set the network in training mode
    net.train()

    # keep track of the loss
    loss_cum = 0
    cnt = 0
    num_correct = 0
    targets = []
    predictions = []

    for i, data in enumerate(loader):

        # sample data
        x, y = data
        x = x.to(device)
        y = y.to(device)

        # set all gradients equal to zero
        net.zero_grad()

        # feed the batch to the network and compute the outputs
        y_pred = net(x)
        #y_pred_sq = np.squeeze(y_pred) # add when using loss_fn: BCEloss()

        pred = torch.round(y_pred.squeeze())  # rounds to the nearest integer

        # compare predictions to true label
        #correct_tensor = pred.eq(y.float().view_as(pred))
        #correct = np.squeeze(correct_tensor.cpu().numpy())
        #num_correct += np.sum(correct)
        targets.extend(y.cpu().detach().numpy().tolist())
        predictions.extend(torch.sigmoid(y_pred).cpu().detach().numpy().tolist())

        #predictions_round = [round(p) for p in predictions]
        #correct_tensor = predictions_round.eq(targets.float().view_as(predictions_round))

        #loss = loss_fn(y_pred_sq, y.float())  #add when using loss_fn: BCEloss()
        loss = loss_fn(y_pred, y.float())

        # Specify L1 and L2 weights
        l1_weight = 0
        l2_weight = 0

        # Compute L1 and L2 loss component
        parameters = []
        for parameter in net.parameters():
            parameters.append(parameter.view(-1))
        l1 = l1_weight * net.compute_l1_loss(torch.cat(parameters))
        l2 = l2_weight * net.compute_l2_loss(torch.cat(parameters))

        # Add L1 and L2 loss components
        loss += l1
        loss += l2

        loss_cum += loss.data.cpu().numpy()
        cnt += 1

        # backpropagate the gradients w.r.t. computed loss
        loss.backward()

        # apply one step in the optimization
        optimizer.step()
        scheduler.step()

        #break #for training one batch

    # compute the average loss
    #mytrain_acc = num_correct
    loss_avg = loss_cum / cnt

    #predictions = np.array(predictions) >= 0.5
    predictions = [1 if p[0] > 0.5 else 0 for p in predictions]
    mytrain_acc  = metrics.accuracy_score(targets, predictions)   *100
    #mytrain_acc = (num_correct/ (len(y)*cnt)) *100
    return loss_avg, mytrain_acc

def loss_fn(outputs, targets):
  return nn.BCEWithLogitsLoss()(outputs, targets.view(-1, 1))

In [ ]:
def train_net_cross(net, train_loader, val_loader, loss_fn, optimizer, epochs , scheduler):

    # transfer the network to the GPU
    net = net.to(device)

    train_loss = np.zeros((epochs))
    test_loss = np.zeros((epochs))
    train_acc = np.zeros((epochs))
    train_acc_alt = np.zeros((epochs))

    test_acc = np.zeros((epochs))
    test_acc_alt = np.zeros((epochs))

    print("Begin training.")

    for epoch in tqdm(range(epochs)):

        # training
        train_loss[epoch], train_acc[epoch] = train_epoch_cross(net, train_loader, loss_fn, optimizer, scheduler)
        #predictions, outputs = eval(net, train_loader)
        #train_acc_alt[epoch] = metrics.accuracy_score(outputs, predictions)

        # validation
        test_loss[epoch], test_acc[epoch] = test_epoch(net, val_loader, loss_fn)
        #predictions, outputs = eval(net, val_loader)
        #test_acc_alt[epoch] = metrics.accuracy_score(outputs, predictions)

        #print('Epoch %5d - Train loss: %.6f - Train accuracy: %.6f - Test loss: %.6f - Test accuracy: %.6f'
        #     % (epoch, train_loss[epoch], train_acc[epoch], test_loss[epoch], test_acc[epoch]))

        #print('Epoch %5d - Train loss: %.6f - Train accuracy: %.6f'
        #    % (epoch, train_loss[epoch], train_acc[epoch]))# for training one batch

    return (train_loss, test_loss), (train_acc , test_acc)
    #return train_loss, train_acc # for training one batch

## Train / test only ones with overlapping values for all predictors

### Import training data

In [ ]:
# import difference for each mutation + labels
dif_mut = pd.read_csv('/content/drive/MyDrive/log_probWT_MUT_Tier1_2_3_common_balanced+-2_2200AA_57maxpool.csv', header = None, names = ['mutation', 'log_difference', 'label'])

# exclude duplicates
#dif_mut.drop_duplicates(subset = 'mutation', inplace =True)

# log_difference has been saved as string of list! Here I convert it back to list of floats
fl_dif = []
for x in dif_mut['log_difference']:
  p = x[1:-1].split(',')
  fl_dif.append([float(i) for i in p])

dif_mut['fl_dif'] = fl_dif

print('Deleterious mutations in dataset: ', len(dif_mut[dif_mut['label'] == 1]))
print('Benign mutations in dataset: ', len(dif_mut[dif_mut['label'] == 0]))


# pad to 2200 AA
N= 2200
fl_dif_pad =[]
for i, mut in dif_mut.iterrows():
  a = mut['fl_dif']
  new_a = a + [0] * (N - len(a))
  fl_dif_pad.append(new_a)
dif_mut['fl_dif_pad'] = fl_dif_pad
dif_mut_shuffled = dif_mut.sample(frac=1)

### Import results of all predictors for DRGN and save them in dictionary

In [ ]:
# Load the CSV file into a Pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/my_colab/3rdYear/GMM/DRGN_mutations_012023_DEOGEN_etc.csv')
# squeeze proteins and mutaitons in one column
df['mutation'] = df['uniprot'] + '_' + df['AA_orig'] + df['position'].astype(str) + df['AA_targ']
df = df.drop(columns=['label'])
# Create a dictionary with the specified columns as key and values
results_all = {}
for key, val1, val2, val3, val4, val5 in zip(df['mutation'], df['DEOGEN2'], df['EVE'], df['FATHMM'], df['SIFT'], df['PolyPhen2']):
    #if key in my_dict: # should be the case since I dont have duplicates
    #    my_dict[key].append([val1, val2, val3, val4, val5])
    #else:
    results_all[key] = [val1, val2, val3, val4, val5]

## keep only the values that have prediction for all predictors
df = df[~df.isnull().any(axis=1)]

### Import testing data (DRGN) representations

In [ ]:
dif_mut_drgn = pd.read_csv('/content/drive/MyDrive/log_probWT_MUT_DRGN_2200AA_57maxpool.csv', header = None, names = ['mutation', 'log_difference', 'label'])
# exclude duplicates
#dif_mut_drgn.drop_duplicates(subset = 'mutation', inplace =True)

dif_mut_drgn_nonan = pd.merge(df, dif_mut_drgn, how='inner', on=['mutation']) # Ti einai auto?? Giati den grafeis sxolia ????? Ok katalaba: Exairw ta DRGN samples pou den exw timi gia olous tous predictors
print(len(dif_mut_drgn_nonan))

# log_difference has been saved as string of list! Here I convert it back to list of floats
fl_dif = []
for x in dif_mut_drgn_nonan['log_difference']:
  p = x[1:-1].split(',')
  fl_dif.append([float(i) for i in p])

dif_mut_drgn_nonan['fl_dif'] = fl_dif

print('Deleterious mutations in dataset: ', len(dif_mut_drgn_nonan[dif_mut_drgn_nonan['label'] == 1]))
print('Benign mutations in dataset: ', len(dif_mut_drgn_nonan[dif_mut_drgn_nonan['label'] == 0]))

# pad to 2200 AA
N= 2200
fl_dif_pad =[]
for i, mut in dif_mut_drgn_nonan.iterrows():
  a = mut['fl_dif']
  new_a = a + [0] * (N - len(a))
  fl_dif_pad.append(new_a)
dif_mut_drgn_nonan['fl_dif_pad'] = fl_dif_pad

stacked_flat_drgn =[]
for i, mut in dif_mut_drgn_nonan.iterrows():
  stacked_flat_drgn.append(torch.tensor(mut['fl_dif_pad']))

dif_mut_drgn_shuffled = dif_mut_drgn_nonan.sample(frac=1)

### Train / test only ones with overlapping values for all predictors


In [ ]:
# Distribution of proteins in trianing / test se tof fold
# Set fixed random number seed
torch.manual_seed(42)
TRAIN_BATCH_SIZE = 64
epochs=200
lr = [3e-5]
h = 4096
hidden2=2048
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

n_folds = 5

sensitivity_D2D, specificity_D2D, mcc_D2D, auc_D2D = {}, {}, {}, {}
sensitivity_DEOGEN2, specificity_DEOGEN2, mcc_DEOGEN2, auc_DEOGEN2 = {}, {}, {}, {}
sensitivity_EVE, specificity_EVE, mcc_EVE, auc_EVE = {}, {}, {}, {}
sensitivity_FATHMM, specificity_FATHMM, mcc_FATHMM, auc_FATHMM = {}, {}, {}, {}
sensitivity_SIFT, specificity_SIFT, mcc_SIFT, auc_SIFT = {}, {}, {}, {}
sensitivity_PolyPhen2, specificity_PolyPhen2, mcc_PolyPhen2, auc_PolyPhen2 = {}, {}, {}, {}

for fold in range(n_folds):
  print(f'Starting {fold} fold')
  test = dif_mut_drgn_shuffled[fold*round(len(dif_mut_drgn_shuffled)/n_folds):(fold+1)*round(len(dif_mut_drgn_shuffled)/n_folds)]
  temp = test.mutation.str.split(pat='_',expand=True)[0]

  stacked_flat_drgn =[]
  for i, mut in test.iterrows():
    stacked_flat_drgn.append(torch.tensor(mut['fl_dif_pad']))

  stacked_drgn = torch.stack(stacked_flat_drgn)
  print('Test set dimensions of fold: ', stacked_drgn.shape)

  cond = dif_mut_shuffled['mutation'].isin(test['mutation'])
  training = dif_mut_shuffled.drop(dif_mut_shuffled[cond].index, inplace = False)

  # exclude test mutations from training data
  stacked_flat =[]
  for i, mut in training.iterrows():
    stacked_flat.append(torch.tensor(mut['fl_dif_pad']))

  stacked_gmm = torch.stack(stacked_flat)
  print('Training set dimensions of fold: ', stacked_gmm.shape)

  training_temp = training.mutation.str.split(pat='_',expand=True)

  # seperate inputs and outputs
  X = stacked_gmm
  y = training['label']

  # Split, shuffle and stratify sequences, mutations, proteins and label
  X_train, X_test, y_train, y_test, mu_all_train, mu_all_test , prot_all_train, prot_all_test = train_test_split(X, y, training_temp[1],  training_temp[0], test_size=0.0002, stratify=y, random_state=69)
  X_train, y_train = np.array(X_train), np.array(y_train)
  X_test, y_test = np.array(X_test), np.array(y_test)

  del stacked_gmm, X
  gc.collect()

  train_dataset = ClassifierDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
  val_dataset = ClassifierDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long())

  train_loader = DataLoader(dataset=train_dataset, batch_size=TRAIN_BATCH_SIZE, drop_last=True)
  val_loader = DataLoader(dataset=val_dataset, batch_size=2, drop_last=True)

  my_net = Classifier2L(h, hidden2, 0.3).to(device)
  my_net.apply(init_weights)
  param_optimizer = list(my_net.named_parameters())
  no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
  optimizer_parameters = [
      {
          "params": [
              p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
          ],
          "weight_decay": 0.001,
      },
      {
          "params": [
              p for n, p in param_optimizer if any(nd in n for nd in no_decay)
          ],
          "weight_decay": 0.02,
      },
  ]

  num_train_steps = int(len(X_train) / TRAIN_BATCH_SIZE * epochs)
  optimizer = torch.optim.AdamW(optimizer_parameters, lr=3e-5)
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=1000, num_training_steps=num_train_steps)

  loss, acc = train_net_cross(my_net, train_loader, val_loader, loss_fn, optimizer, epochs, scheduler )
  labels_drgn = test['label'].tolist()
  X_drgn, y_drgn = np.array(stacked_drgn), np.array(labels_drgn)  # Actual test

  drgn_dataset = ClassifierDataset(torch.from_numpy(X_drgn).float(), torch.from_numpy(y_drgn).long())
  drgn_loader = DataLoader(dataset=drgn_dataset, batch_size=1 , drop_last=True)

  y_pred_list = []
  predictions_drgn= []
  my_net.eval()

  with torch.no_grad():
      for X_batch, _ in drgn_loader:
          X_batch = X_batch.to(device)
          y_test_pred = my_net(X_batch)
          predictions_drgn.extend(torch.sigmoid(y_test_pred).cpu().detach().numpy().tolist())

  flat_list = []
  for sublist in predictions_drgn:
      for item in sublist:
          flat_list.append(item)

  predictions_drgn_D2D = [1 if i>=0.5 else 0 for i in flat_list]
  confusion_matrix_df = pd.DataFrame(confusion_matrix(y_drgn, predictions_drgn_D2D))
  auc = metrics.roc_auc_score(y_drgn[:len(predictions_drgn_D2D)], flat_list)
  tn, fp, fn, tp = confusion_matrix(y_drgn, predictions_drgn_D2D).ravel()
  TPR =tp / (tp + fn)
  TNR = tn / (tn + fp)

  #accuracy_D2D[fold]=((tp+tn)/(tp +tn+ fp+ fn))*100
  sensitivity_D2D[fold]=TPR*100
  specificity_D2D[fold]=TNR*100
  mcc_D2D[fold]= matthews_corrcoef(y_drgn, predictions_drgn_D2D)*100
  auc_D2D[fold] =round(auc, 3)*100


  # DEOGEN2 results for fold
  DEOGEN2_drgn = []
  for mut_tet in test['mutation'].values:
    DEOGEN2_drgn.append(results_all[mut_tet][0])

  ## use when NA --> errors
  DEOGEN2_drgn_Na = []
  for j,i in enumerate(DEOGEN2_drgn):
    if notNaN(i): # if not NaN
      DEOGEN2_drgn_Na.append(i)
    else:
      DEOGEN2_drgn_Na.append(1- y_drgn[j])
  predictions_drgn_DEOGEN2 = [1 if i>0.5 else 0 for i in DEOGEN2_drgn_Na]

  confusion_matrix_df = pd.DataFrame(confusion_matrix(y_drgn, predictions_drgn_DEOGEN2))
  auc = metrics.roc_auc_score(y_drgn[:len(predictions_drgn_DEOGEN2)], DEOGEN2_drgn_Na )
  tn, fp, fn, tp = confusion_matrix(y_drgn, predictions_drgn_DEOGEN2).ravel()

  ## use when NA --> omit
  #y_drgn_DEOGEN2 = [x for x, y in zip(y_drgn, DEOGEN2_drgn) if not pd.isna(y)]
  #predictions_drgn_DEOGEN2 = [x for x in DEOGEN2_drgn if not pd.isna(x)]
  #predictions_drgn_DEOGEN2 = [1 if i>0.5 else 0 for i in predictions_drgn_DEOGEN2]
  #confusion_matrix_df = pd.DataFrame(confusion_matrix(y_drgn_DEOGEN2, predictions_drgn_DEOGEN2))
  #auc = metrics.roc_auc_score(y_drgn_DEOGEN2[:len(predictions_drgn_DEOGEN2)], predictions_drgn_DEOGEN2 )
  #tn, fp, fn, tp = confusion_matrix(y_drgn_DEOGEN2, predictions_drgn_DEOGEN2).ravel()

  TPR =tp / (tp + fn)
  TNR = tn / (tn + fp)

  #accuracy_DEOGEN2[fold]=((tp+tn)/(tp +tn+ fp+ fn))*100
  sensitivity_DEOGEN2[fold]=TPR*100
  specificity_DEOGEN2[fold]=TNR*100
  mcc_DEOGEN2[fold]= matthews_corrcoef(y_drgn, predictions_drgn_DEOGEN2)*100 # y_drgn_DEOGEN2 when omit
  auc_DEOGEN2[fold] =round(auc, 3)*100


  # EVE results for fold
  EVE_drgn = []
  for mut_tet in test['mutation'].values:
    EVE_drgn.append(results_all[mut_tet][1])

  # use when NA --> errors
  #EVE_drgn_Na = []
  #for j,i in enumerate(EVE_drgn):
  #  if notNaN(i): # if not NaN
  #    if i>=0.63:
  #      temp.append(1)
  #    elif i<0.38:
   #     temp.append(0)
   # else:
   #   EVE_drgn_Na.append(1- y_drgn[j])
  #predictions_drgn_EVE = [1 if i>=0.5 else 0 for i in EVE_drgn_Na ]
  temp=[]
  for i in EVE_drgn:
    if i<=0.62 and i>=0.5:
      temp.append(i-0.13)
    elif i>=0.38 and i<0.5:
      temp.append(i+0.17)
    else:
      temp.append(i)
  predictions_drgn_EVE = [1 if i>0.5 else 0 for i in temp]
  confusion_matrix_df = pd.DataFrame(confusion_matrix(y_drgn, predictions_drgn_EVE))
  auc = metrics.roc_auc_score(y_drgn[:len(predictions_drgn_EVE)], temp) # EVE_drgn_Na
  tn, fp, fn, tp = confusion_matrix(y_drgn, predictions_drgn_EVE).ravel()

  # use when NA --> omit
  #y_drgn_EVE = [x for x, y in zip(y_drgn, EVE_drgn) if not pd.isna(y)]
  #predictions_drgn_EVE = [x for x in EVE_drgn if not pd.isna(x)]
  #temp = []
  #for j,i in enumerate(predictions_drgn_EVE):
  #  if i>=0.63:
  #    temp.append(1)
  #  elif i<0.38:
  #    temp.append(0)
  #  else:
  #    temp.append(1- y_drgn[j])
  #predictions_drgn_EVE = temp
  #confusion_matrix_df = pd.DataFrame(confusion_matrix(y_drgn_EVE, predictions_drgn_EVE))
  #auc = metrics.roc_auc_score(y_drgn_EVE[:len(predictions_drgn_EVE)], predictions_drgn_EVE) # EVE_drgn_Na
  #tn, fp, fn, tp = confusion_matrix(y_drgn_EVE, predictions_drgn_EVE).ravel()

  TPR =tp / (tp + fn)
  TNR = tn / (tn + fp)

  #accuracy_EVE[fold]=((tp+tn)/(tp +tn+ fp+ fn))*100
  sensitivity_EVE[fold]=TPR*100
  specificity_EVE[fold]=TNR*100
  mcc_EVE[fold]= matthews_corrcoef(y_drgn, predictions_drgn_EVE)*100 # y_drgn_EVE when omit
  auc_EVE[fold] =round(auc, 3)*100

  # FATHMM results for fold
  FATHMM_drgn = []
  for mut_tet in test['mutation'].values:
    FATHMM_drgn.append(results_all[mut_tet][2])
  # use when NA --> errors
  #FATHMM_drgn_Na = []
  #for j,i in enumerate(FATHMM_drgn):
  #  if notNaN(i): # if not NaN
  #    FATHMM_drgn_Na.append(i)
  #  else:
  #    FATHMM_drgn_Na.append(1- y_drgn[j])
  #predictions_drgn_FATHMM = [1 if i<-0.75 else 0 for i in FATHMM_drgn_Na ]

  # use when NA --> omit
  y_drgn_FATHMM = [x for x, y in zip(y_drgn, FATHMM_drgn) if not pd.isna(y)]
  predictions_drgn_FATHMM = [x for x in FATHMM_drgn if not pd.isna(x)]
  predictions_drgn_FATHMM = [1 if i<-0.75 else 0 for i in predictions_drgn_FATHMM]

  confusion_matrix_df = pd.DataFrame(confusion_matrix(y_drgn_FATHMM, predictions_drgn_FATHMM))
  auc = metrics.roc_auc_score(y_drgn_FATHMM[:len(predictions_drgn_FATHMM)], predictions_drgn_FATHMM ) # FATHMM_drgn_Na
  tn, fp, fn, tp = confusion_matrix(y_drgn_FATHMM, predictions_drgn_FATHMM).ravel()
  TPR =tp / (tp + fn)
  TNR = tn / (tn + fp)

  #accuracy_FATHMM[fold]=((tp+tn)/(tp +tn+ fp+ fn))*100
  sensitivity_FATHMM[fold]=TPR*100
  specificity_FATHMM[fold]=TNR*100
  mcc_FATHMM[fold]= matthews_corrcoef(y_drgn_FATHMM, predictions_drgn_FATHMM)*100
  auc_FATHMM[fold] =round(auc, 3)*100


  # SIFT results for fold
  SIFT_drgn = []
  for mut_tet in test['mutation'].values:
    SIFT_drgn.append(results_all[mut_tet][3])

  # use when NA --> errors
  #SIFT_drgn_Na = []
  #for j,i in enumerate(SIFT_drgn):
  #  if notNaN(i): # if not NaN
  #    SIFT_drgn_Na.append(i)
  #  else:
  #    SIFT_drgn_Na.append(1- y_drgn[j])
  #predictions_drgn_SIFT = [1 if i<=0.05 else 0 for i in SIFT_drgn_Na ]

  # use when NA --> omit
  y_drgn_SIFT = [x for x, y in zip(y_drgn, SIFT_drgn) if not pd.isna(y)]
  predictions_drgn_SIFT = [x for x in SIFT_drgn if not pd.isna(x)]
  predictions_drgn_SIFT = [1 if i<=0.05 else 0 for i in predictions_drgn_SIFT]

  confusion_matrix_df = pd.DataFrame(confusion_matrix(y_drgn_SIFT, predictions_drgn_SIFT))
  auc = metrics.roc_auc_score(y_drgn_SIFT[:len(predictions_drgn_SIFT)], predictions_drgn_SIFT ) #SIFT_drgn_Na
  tn, fp, fn, tp = confusion_matrix(y_drgn_SIFT, predictions_drgn_SIFT).ravel()
  TPR =tp / (tp + fn)
  TNR = tn / (tn + fp)

  #accuracy_SIFT[fold]=((tp+tn)/(tp +tn+ fp+ fn))*100
  sensitivity_SIFT[fold]=TPR*100
  specificity_SIFT[fold]=TNR*100
  mcc_SIFT[fold]= matthews_corrcoef(y_drgn_SIFT, predictions_drgn_SIFT)*100
  auc_SIFT[fold] =round(auc, 3)*100


  # PolyPhen2 results for fold
  PolyPhen2_drgn = []
  for mut_tet in test['mutation'].values:
    PolyPhen2_drgn.append(results_all[mut_tet][4])

  # use when NA --> errors
  #PolyPhen2_drgn_Na = []
  #for j,i in enumerate(PolyPhen2_drgn):
  #  if notNaN(i): # if not NaN
  #    PolyPhen2_drgn_Na.append(i)
  #  else:
  #    PolyPhen2_drgn_Na.append(1- y_drgn[j])
  #predictions_drgn_PolyPhen2 = [1 if i>=0.49 else 0 for i in PolyPhen2_drgn_Na]

  # use when NA --> omit
  y_drgn_PolyPhen2 = [x for x, y in zip(y_drgn, PolyPhen2_drgn) if not pd.isna(y)]
  predictions_drgn_PolyPhen2 = [x for x in PolyPhen2_drgn if not pd.isna(x)]
  predictions_drgn_PolyPhen2 = [1 if i>=0.49 else 0 for i in predictions_drgn_PolyPhen2]
  confusion_matrix_df = pd.DataFrame(confusion_matrix(y_drgn_PolyPhen2, predictions_drgn_PolyPhen2))
  auc = metrics.roc_auc_score(y_drgn[:len(predictions_drgn_PolyPhen2)],predictions_drgn_PolyPhen2 ) # PolyPhen2_drgn_Na
  tn, fp, fn, tp = confusion_matrix(y_drgn_PolyPhen2, predictions_drgn_PolyPhen2).ravel()
  TPR =tp / (tp + fn)
  TNR = tn / (tn + fp)

  #accuracy_PolyPhen2[fold]=((tp+tn)/(tp +tn+ fp+ fn))*100
  sensitivity_PolyPhen2[fold]=TPR*100
  specificity_PolyPhen2[fold]=TNR*100
  mcc_PolyPhen2[fold]= matthews_corrcoef(y_drgn_PolyPhen2, predictions_drgn_PolyPhen2)*100
  auc_PolyPhen2[fold] =round(auc, 3)*100

In [ ]:
ds=[sensitivity_D2D,  specificity_D2D,  mcc_D2D, auc_D2D] # list of dictionaries
test = pd.DataFrame(columns = ['metric', 'fold', 'value'])
fold_list, value_list = [],[]
for i, m in enumerate(ds):
  for key, value in m.items():
    fold_list.append(key)
    value_list.append(value)

test['metric'] = ['sensitivity']* len(sensitivity_D2D) + ['specificity']* len(sensitivity_D2D)+  ['mcc']* len(sensitivity_D2D)+  ['auc']* len(sensitivity_D2D)
test['fold'] = fold_list
test['value'] = value_list
test['predictor'] = ['D2D']* len(sensitivity_D2D) * len(ds)

ds=[sensitivity_DEOGEN2,  specificity_DEOGEN2,  mcc_DEOGEN2, auc_DEOGEN2] # list of dictionaries
df2 = pd.DataFrame(columns = ['metric', 'fold', 'value'])
fold_list, value_list = [],[]
for i, m in enumerate(ds):
  for key, value in m.items():
    fold_list.append(key)
    value_list.append(value)

df2['metric'] = ['sensitivity']* len(sensitivity_DEOGEN2) + ['specificity']* len(sensitivity_DEOGEN2)+  ['mcc']* len(sensitivity_DEOGEN2)+  ['auc']* len(sensitivity_DEOGEN2)
df2['fold'] = fold_list
df2['value'] = value_list
df2['predictor'] = ['DEOGEN2']* len(sensitivity_D2D) * len(ds)
test = test.append(df2, ignore_index=True)

ds=[sensitivity_EVE,  specificity_EVE,  mcc_EVE, auc_EVE] # list of dictionaries
df2 = pd.DataFrame(columns = ['metric', 'fold', 'value'])
fold_list, value_list = [],[]
for i, m in enumerate(ds):
  for key, value in m.items():
    fold_list.append(key)
    value_list.append(value)

df2['metric'] = ['sensitivity']* len(sensitivity_EVE) + ['specificity']* len(sensitivity_EVE)+  ['mcc']* len(sensitivity_EVE)+  ['auc']* len(sensitivity_EVE)
df2['fold'] = fold_list
df2['value'] = value_list
df2['predictor'] = ['EVE']* len(sensitivity_D2D) * len(ds)
test = test.append(df2, ignore_index=True)

ds=[sensitivity_FATHMM,  specificity_FATHMM,  mcc_FATHMM, auc_FATHMM] # list of dictionaries
df2 = pd.DataFrame(columns = ['metric', 'fold', 'value'])
fold_list, value_list = [],[]
for i, m in enumerate(ds):
  for key, value in m.items():
    fold_list.append(key)
    value_list.append(value)

df2['metric'] = ['sensitivity']* len(sensitivity_FATHMM) + ['specificity']* len(sensitivity_FATHMM)+  ['mcc']* len(sensitivity_FATHMM)+  ['auc']* len(sensitivity_FATHMM)
df2['fold'] = fold_list
df2['value'] = value_list
df2['predictor'] = ['FATHMM']* len(sensitivity_D2D) * len(ds)
test = test.append(df2, ignore_index=True)

ds=[sensitivity_SIFT,  specificity_SIFT,  mcc_SIFT, auc_SIFT] # list of dictionaries
df2 = pd.DataFrame(columns = ['metric', 'fold', 'value'])
fold_list, value_list = [],[]
for i, m in enumerate(ds):
  for key, value in m.items():
    fold_list.append(key)
    value_list.append(value)

df2['metric'] = ['sensitivity']* len(sensitivity_SIFT) + ['specificity']* len(sensitivity_SIFT)+  ['mcc']* len(sensitivity_SIFT)+  ['auc']* len(sensitivity_SIFT)
df2['fold'] = fold_list
df2['value'] = value_list
df2['predictor'] = ['SIFT']* len(sensitivity_D2D) * len(ds)
test = test.append(df2, ignore_index=True)

ds=[sensitivity_PolyPhen2,  specificity_PolyPhen2,  mcc_PolyPhen2, auc_PolyPhen2] # list of dictionaries
df2 = pd.DataFrame(columns = ['metric', 'fold', 'value'])
fold_list, value_list = [],[]
for i, m in enumerate(ds):
  for key, value in m.items():
    fold_list.append(key)
    value_list.append(value)

df2['metric'] = ['sensitivity']* len(sensitivity_PolyPhen2) + ['specificity']* len(sensitivity_PolyPhen2)+  ['mcc']* len(sensitivity_PolyPhen2)+  ['auc']* len(sensitivity_PolyPhen2)
df2['fold'] = fold_list
df2['value'] = value_list
df2['predictor'] = ['PolyPhen2']* len(sensitivity_D2D) * len(ds)
test = test.append(df2, ignore_index=True)

#fig = px.violin(test, y="metric", x="value", color="metric", box=True, points="all", width=800, height=400, title="5 folds D2D")
#fig.update_xaxes(range=[-60, 100])
#fig.show()
test_notmcc = test[test['metric'] != 'mcc']

In [ ]:
fig = px.box(test_notmcc, x="metric", y="value", color="predictor", color_discrete_map = {'D2D': '#0E6A7B', 'EVE': '#ffc907', 'FATHMM': '#56B4E9','PolyPhen2': '#82ad27','DEOGEN2': '#695647', 'SIFT':'#4234a4'}, width=1600, height=500) #2292a7
#fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.update_layout({
'plot_bgcolor': 'rgba(255, 255, 255, 255)' ,
'paper_bgcolor': 'rgba(255, 255, 255, 255)',
})

fig.show()

In [ ]:
fig = px.box(test_notmcc, x="metric", y="value", color="predictor", width=1600, height=500)
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()

In [ ]:
test_mcc = test[test['metric'] == 'mcc']
fig = px.box(test_mcc, x="metric", y="value", color="predictor", color_discrete_map = {'D2D': '#0E6A7B', 'EVE': '#ffc907', 'FATHMM': '#56B4E9','PolyPhen2': '#82ad27','DEOGEN2': '#695647', 'SIFT':'#4234a4'}, width=600, height=500) # #2292a7
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.update_layout({
'plot_bgcolor': 'rgba(255, 255, 255, 255)' ,
'paper_bgcolor': 'rgba(255, 255, 255, 255)',
})
fig.show()

## When missing values considered errors

### Import training data

In [ ]:
# import difference for each mutation + labels
dif_mut = pd.read_csv('/content/drive/MyDrive/log_probWT_MUT_Tier1_2_3_common_balanced+-2_2200AA_57maxpool.csv', header = None, names = ['mutation', 'log_difference', 'label'])

# exclude duplicates
#dif_mut.drop_duplicates(subset = 'mutation', inplace =True)

# log_difference has been saved as string of list! Here I convert it back to list of floats
fl_dif = []
for x in dif_mut['log_difference']:
  p = x[1:-1].split(',')
  fl_dif.append([float(i) for i in p])

dif_mut['fl_dif'] = fl_dif

print('Deleterious mutations in dataset: ', len(dif_mut[dif_mut['label'] == 1]))
print('Benign mutations in dataset: ', len(dif_mut[dif_mut['label'] == 0]))


# pad to 2200 AA
N= 2200
fl_dif_pad =[]
for i, mut in dif_mut.iterrows():
  a = mut['fl_dif']
  new_a = a + [0] * (N - len(a))
  fl_dif_pad.append(new_a)
dif_mut['fl_dif_pad'] = fl_dif_pad
dif_mut_shuffled = dif_mut.sample(frac=1)

### Import results of all predictors for DRGN and save them in dictionary

In [ ]:
# Load the CSV file into a Pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/my_colab/3rdYear/GMM/DRGN_mutations_012023_DEOGEN_etc.csv')
# squeeze proteins and mutaitons in one column
df['mutation'] = df['uniprot'] + '_' + df['AA_orig'] + df['position'].astype(str) + df['AA_targ']
df = df.drop(columns=['label'])
# Create a dictionary with the specified columns as key and values
results_all = {}
for key, val1, val2, val3, val4, val5 in zip(df['mutation'], df['DEOGEN2'], df['EVE'], df['FATHMM'], df['SIFT'], df['PolyPhen2']):
    #if key in my_dict: # should be the case since I dont have duplicates
    #    my_dict[key].append([val1, val2, val3, val4, val5])
    #else:
    results_all[key] = [val1, val2, val3, val4, val5]

## keep only the values that have prediction for all predictors
#df = df[~df.isnull().any(axis=1)]

### Import testing data (DRGN) representations

In [ ]:
dif_mut_drgn = pd.read_csv('/content/drive/MyDrive/log_probWT_MUT_DRGN_2200AA_57maxpool.csv', header = None, names = ['mutation', 'log_difference', 'label'])
# exclude duplicates
#dif_mut_drgn.drop_duplicates(subset = 'mutation', inplace =True)

dif_mut_drgn_nonan = pd.merge(df, dif_mut_drgn, how='inner', on=['mutation']) # Ti einai auto?? Giati den grafeis sxolia ????? Ok katalaba: Exairw ta DRGN samples pou den exw timi gia olous tous predictors
print(len(dif_mut_drgn_nonan))

# log_difference has been saved as string of list! Here I convert it back to list of floats
fl_dif = []
for x in dif_mut_drgn_nonan['log_difference']:
  p = x[1:-1].split(',')
  fl_dif.append([float(i) for i in p])

dif_mut_drgn_nonan['fl_dif'] = fl_dif

print('Deleterious mutations in dataset: ', len(dif_mut_drgn_nonan[dif_mut_drgn_nonan['label'] == 1]))
print('Benign mutations in dataset: ', len(dif_mut_drgn_nonan[dif_mut_drgn_nonan['label'] == 0]))

# pad to 2200 AA
N= 2200
fl_dif_pad =[]
for i, mut in dif_mut_drgn_nonan.iterrows():
  a = mut['fl_dif']
  new_a = a + [0] * (N - len(a))
  fl_dif_pad.append(new_a)
dif_mut_drgn_nonan['fl_dif_pad'] = fl_dif_pad

stacked_flat_drgn =[]
for i, mut in dif_mut_drgn_nonan.iterrows():
  stacked_flat_drgn.append(torch.tensor(mut['fl_dif_pad']))

dif_mut_drgn_shuffled = dif_mut_drgn_nonan.sample(frac=1)

### Train /test

In [ ]:
# Distribution of proteins in trianing / test se tof fold
# Set fixed random number seed
torch.manual_seed(42)
TRAIN_BATCH_SIZE = 64
epochs=200
lr = [3e-5]
h = 4096
hidden2=2048
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

n_folds = 5

sensitivity_D2D, specificity_D2D, mcc_D2D, auc_D2D = {}, {}, {}, {}
sensitivity_DEOGEN2, specificity_DEOGEN2, mcc_DEOGEN2, auc_DEOGEN2 = {}, {}, {}, {}
sensitivity_EVE, specificity_EVE, mcc_EVE, auc_EVE = {}, {}, {}, {}
sensitivity_FATHMM, specificity_FATHMM, mcc_FATHMM, auc_FATHMM = {}, {}, {}, {}
sensitivity_SIFT, specificity_SIFT, mcc_SIFT, auc_SIFT = {}, {}, {}, {}
sensitivity_PolyPhen2, specificity_PolyPhen2, mcc_PolyPhen2, auc_PolyPhen2 = {}, {}, {}, {}

for fold in range(n_folds):
  print(f'Starting {fold} fold')
  test = dif_mut_drgn_shuffled[fold*round(len(dif_mut_drgn_shuffled)/n_folds):(fold+1)*round(len(dif_mut_drgn_shuffled)/n_folds)]
  temp = test.mutation.str.split(pat='_',expand=True)[0]

  stacked_flat_drgn =[]
  for i, mut in test.iterrows():
    stacked_flat_drgn.append(torch.tensor(mut['fl_dif_pad']))

  stacked_drgn = torch.stack(stacked_flat_drgn)
  print('Test set dimensions of fold: ', stacked_drgn.shape)

  cond = dif_mut_shuffled['mutation'].isin(test['mutation'])
  training = dif_mut_shuffled.drop(dif_mut_shuffled[cond].index, inplace = False)

  # exclude test mutations from training data
  stacked_flat =[]
  for i, mut in training.iterrows():
    stacked_flat.append(torch.tensor(mut['fl_dif_pad']))

  stacked_gmm = torch.stack(stacked_flat)
  print('Training set dimensions of fold: ', stacked_gmm.shape)

  training_temp = training.mutation.str.split(pat='_',expand=True)

  # seperate inputs and outputs
  X = stacked_gmm
  y = training['label']

  # Split, shuffle and stratify sequences, mutations, proteins and label
  X_train, X_test, y_train, y_test, mu_all_train, mu_all_test , prot_all_train, prot_all_test = train_test_split(X, y, training_temp[1],  training_temp[0], test_size=0.0002, stratify=y, random_state=69)
  X_train, y_train = np.array(X_train), np.array(y_train)
  X_test, y_test = np.array(X_test), np.array(y_test)

  del stacked_gmm, X
  gc.collect()

  train_dataset = ClassifierDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
  val_dataset = ClassifierDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long())

  train_loader = DataLoader(dataset=train_dataset, batch_size=TRAIN_BATCH_SIZE, drop_last=True)
  val_loader = DataLoader(dataset=val_dataset, batch_size=2, drop_last=True)

  my_net = Classifier2L(h, hidden2, 0.3).to(device)
  my_net.apply(init_weights)
  param_optimizer = list(my_net.named_parameters())
  no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
  optimizer_parameters = [
      {
          "params": [
              p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
          ],
          "weight_decay": 0.001,
      },
      {
          "params": [
              p for n, p in param_optimizer if any(nd in n for nd in no_decay)
          ],
          "weight_decay": 0.02,
      },
  ]

  num_train_steps = int(len(X_train) / TRAIN_BATCH_SIZE * epochs)
  optimizer = torch.optim.AdamW(optimizer_parameters, lr=3e-5)
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=1000, num_training_steps=num_train_steps)

  loss, acc = train_net_cross(my_net, train_loader, val_loader, loss_fn, optimizer, epochs, scheduler )
  labels_drgn = test['label'].tolist()
  X_drgn, y_drgn = np.array(stacked_drgn), np.array(labels_drgn)  # Actual test

  drgn_dataset = ClassifierDataset(torch.from_numpy(X_drgn).float(), torch.from_numpy(y_drgn).long())
  drgn_loader = DataLoader(dataset=drgn_dataset, batch_size=1 , drop_last=True)

  y_pred_list = []
  predictions_drgn= []
  my_net.eval()

  with torch.no_grad():
      for X_batch, _ in drgn_loader:
          X_batch = X_batch.to(device)
          y_test_pred = my_net(X_batch)
          predictions_drgn.extend(torch.sigmoid(y_test_pred).cpu().detach().numpy().tolist())

  flat_list = []
  for sublist in predictions_drgn:
      for item in sublist:
          flat_list.append(item)

  predictions_drgn_D2D = [1 if i>=0.5 else 0 for i in flat_list]
  confusion_matrix_df = pd.DataFrame(confusion_matrix(y_drgn, predictions_drgn_D2D))
  auc = metrics.roc_auc_score(y_drgn[:len(predictions_drgn_D2D)], flat_list)
  tn, fp, fn, tp = confusion_matrix(y_drgn, predictions_drgn_D2D).ravel()
  TPR =tp / (tp + fn)
  TNR = tn / (tn + fp)

  #accuracy_D2D[fold]=((tp+tn)/(tp +tn+ fp+ fn))*100
  sensitivity_D2D[fold]=TPR*100
  specificity_D2D[fold]=TNR*100
  mcc_D2D[fold]= matthews_corrcoef(y_drgn, predictions_drgn_D2D)*100
  auc_D2D[fold] =round(auc, 3)*100


  # DEOGEN2 results for fold
  DEOGEN2_drgn = []
  for mut_tet in test['mutation'].values:
    DEOGEN2_drgn.append(results_all[mut_tet][0])

  ## use when NA --> errors
  DEOGEN2_drgn_Na = []
  for j,i in enumerate(DEOGEN2_drgn):
    if notNaN(i): # if not NaN
      DEOGEN2_drgn_Na.append(i)
    else:
      DEOGEN2_drgn_Na.append(1- y_drgn[j])
  predictions_drgn_DEOGEN2 = [1 if i>0.5 else 0 for i in DEOGEN2_drgn_Na]

  confusion_matrix_df = pd.DataFrame(confusion_matrix(y_drgn, predictions_drgn_DEOGEN2))
  auc = metrics.roc_auc_score(y_drgn[:len(predictions_drgn_DEOGEN2)], DEOGEN2_drgn_Na )
  tn, fp, fn, tp = confusion_matrix(y_drgn, predictions_drgn_DEOGEN2).ravel()

  ## use when NA --> omit
  #y_drgn_DEOGEN2 = [x for x, y in zip(y_drgn, DEOGEN2_drgn) if not pd.isna(y)]
  #predictions_drgn_DEOGEN2 = [x for x in DEOGEN2_drgn if not pd.isna(x)]
  #predictions_drgn_DEOGEN2 = [1 if i>0.5 else 0 for i in predictions_drgn_DEOGEN2]
  #confusion_matrix_df = pd.DataFrame(confusion_matrix(y_drgn_DEOGEN2, predictions_drgn_DEOGEN2))
  #auc = metrics.roc_auc_score(y_drgn_DEOGEN2[:len(predictions_drgn_DEOGEN2)], predictions_drgn_DEOGEN2 )
  #tn, fp, fn, tp = confusion_matrix(y_drgn_DEOGEN2, predictions_drgn_DEOGEN2).ravel()

  TPR =tp / (tp + fn)
  TNR = tn / (tn + fp)

  #accuracy_DEOGEN2[fold]=((tp+tn)/(tp +tn+ fp+ fn))*100
  sensitivity_DEOGEN2[fold]=TPR*100
  specificity_DEOGEN2[fold]=TNR*100
  mcc_DEOGEN2[fold]= matthews_corrcoef(y_drgn, predictions_drgn_DEOGEN2)*100 # y_drgn_DEOGEN2 when omit
  auc_DEOGEN2[fold] =round(auc, 3)*100


  # EVE results for fold
  EVE_drgn = []
  for mut_tet in test['mutation'].values:
    EVE_drgn.append(results_all[mut_tet][1])


  # use when NA --> errors
  EVE_drgn_Na = []
  for j,i in enumerate(EVE_drgn):
    if notNaN(i): # if not NaN
      EVE_drgn_Na.append(i)
    else:
      EVE_drgn_Na.append(1- y_drgn[j])
  predictions_drgn_EVE = [1 if i>0.5 else 0 for i in EVE_drgn_Na]

  confusion_matrix_df = pd.DataFrame(confusion_matrix(y_drgn, predictions_drgn_EVE))
  auc = metrics.roc_auc_score(y_drgn[:len(predictions_drgn_EVE)], EVE_drgn_Na) # EVE_drgn_Na
  tn, fp, fn, tp = confusion_matrix(y_drgn, predictions_drgn_EVE).ravel()

  # use when NA --> omit
  #y_drgn_EVE = [x for x, y in zip(y_drgn, EVE_drgn) if not pd.isna(y)]
  #predictions_drgn_EVE = [x for x in EVE_drgn if not pd.isna(x)]
  #temp = []
  #for j,i in enumerate(predictions_drgn_EVE):
  #  if i>=0.63:
  #    temp.append(1)
  #  elif i<0.38:
  #    temp.append(0)
  #  else:
  #    temp.append(1- y_drgn[j])
  #predictions_drgn_EVE = temp
  #confusion_matrix_df = pd.DataFrame(confusion_matrix(y_drgn_EVE, predictions_drgn_EVE))
  #auc = metrics.roc_auc_score(y_drgn_EVE[:len(predictions_drgn_EVE)], predictions_drgn_EVE) # EVE_drgn_Na
  #tn, fp, fn, tp = confusion_matrix(y_drgn_EVE, predictions_drgn_EVE).ravel()

  TPR =tp / (tp + fn)
  TNR = tn / (tn + fp)

  #accuracy_EVE[fold]=((tp+tn)/(tp +tn+ fp+ fn))*100
  sensitivity_EVE[fold]=TPR*100
  specificity_EVE[fold]=TNR*100
  mcc_EVE[fold]= matthews_corrcoef(y_drgn, predictions_drgn_EVE)*100 # y_drgn_EVE when omit
  auc_EVE[fold] =round(auc, 3)*100

  # FATHMM results for fold
  FATHMM_drgn = []
  for mut_tet in test['mutation'].values:
    FATHMM_drgn.append(results_all[mut_tet][2])

  # use when NA --> errors
  FATHMM_drgn_Na = []
  for j,i in enumerate(FATHMM_drgn):
    if notNaN(i): # if not NaN
      FATHMM_drgn_Na.append(i)
    else:
      FATHMM_drgn_Na.append(1- y_drgn[j])
  predictions_drgn_FATHMM = [1 if i<-0.75 else 0 for i in FATHMM_drgn_Na ]

  # use when NA --> omit
  #y_drgn_FATHMM = [x for x, y in zip(y_drgn, FATHMM_drgn) if not pd.isna(y)]
  #predictions_drgn_FATHMM = [x for x in FATHMM_drgn if not pd.isna(x)]
  #predictions_drgn_FATHMM = [1 if i<-0.75 else 0 for i in predictions_drgn_FATHMM]

  confusion_matrix_df = pd.DataFrame(confusion_matrix(y_drgn, predictions_drgn_FATHMM))
  auc = metrics.roc_auc_score(y_drgn[:len(predictions_drgn_FATHMM)], FATHMM_drgn_Na ) # FATHMM_drgn_Na when omit
  tn, fp, fn, tp = confusion_matrix(y_drgn, predictions_drgn_FATHMM).ravel()
  TPR =tp / (tp + fn)
  TNR = tn / (tn + fp)

  #accuracy_FATHMM[fold]=((tp+tn)/(tp +tn+ fp+ fn))*100
  sensitivity_FATHMM[fold]=TPR*100
  specificity_FATHMM[fold]=TNR*100
  mcc_FATHMM[fold]= matthews_corrcoef(y_drgn, predictions_drgn_FATHMM)*100
  auc_FATHMM[fold] =round(auc, 3)*100


  # SIFT results for fold
  SIFT_drgn = []
  for mut_tet in test['mutation'].values:
    SIFT_drgn.append(results_all[mut_tet][3])

  # use when NA --> errors
  SIFT_drgn_Na = []
  for j,i in enumerate(SIFT_drgn):
    if notNaN(i): # if not NaN
      SIFT_drgn_Na.append(i)
    else:
      SIFT_drgn_Na.append(1- y_drgn[j])
  predictions_drgn_SIFT = [1 if i<=0.05 else 0 for i in SIFT_drgn_Na ]

  # use when NA --> omit
  #y_drgn_SIFT = [x for x, y in zip(y_drgn, SIFT_drgn) if not pd.isna(y)]
  #predictions_drgn_SIFT = [x for x in SIFT_drgn if not pd.isna(x)]
  #predictions_drgn_SIFT = [1 if i<=0.05 else 0 for i in predictions_drgn_SIFT]

  confusion_matrix_df = pd.DataFrame(confusion_matrix(y_drgn, predictions_drgn_SIFT))
  auc = metrics.roc_auc_score(y_drgn[:len(predictions_drgn_SIFT)], SIFT_drgn_Na ) #SIFT_drgn_Na
  tn, fp, fn, tp = confusion_matrix(y_drgn, predictions_drgn_SIFT).ravel()
  TPR =tp / (tp + fn)
  TNR = tn / (tn + fp)

  #accuracy_SIFT[fold]=((tp+tn)/(tp +tn+ fp+ fn))*100
  sensitivity_SIFT[fold]=TPR*100
  specificity_SIFT[fold]=TNR*100
  mcc_SIFT[fold]= matthews_corrcoef(y_drgn, predictions_drgn_SIFT)*100
  auc_SIFT[fold] =round(auc, 3)*100


  # PolyPhen2 results for fold
  PolyPhen2_drgn = []
  for mut_tet in test['mutation'].values:
    PolyPhen2_drgn.append(results_all[mut_tet][4])

  # use when NA --> errors
  PolyPhen2_drgn_Na = []
  for j,i in enumerate(PolyPhen2_drgn):
    if notNaN(i): # if not NaN
      PolyPhen2_drgn_Na.append(i)
    else:
      PolyPhen2_drgn_Na.append(1- y_drgn[j])
  predictions_drgn_PolyPhen2 = [1 if i>=0.49 else 0 for i in PolyPhen2_drgn_Na]

  # use when NA --> omit
  #y_drgn_PolyPhen2 = [x for x, y in zip(y_drgn, PolyPhen2_drgn) if not pd.isna(y)]
  #predictions_drgn_PolyPhen2 = [x for x in PolyPhen2_drgn if not pd.isna(x)]
  #predictions_drgn_PolyPhen2 = [1 if i>=0.49 else 0 for i in predictions_drgn_PolyPhen2]

  confusion_matrix_df = pd.DataFrame(confusion_matrix(y_drgn, predictions_drgn_PolyPhen2))
  auc = metrics.roc_auc_score(y_drgn[:len(predictions_drgn_PolyPhen2)],PolyPhen2_drgn_Na ) # PolyPhen2_drgn_Na
  tn, fp, fn, tp = confusion_matrix(y_drgn, predictions_drgn_PolyPhen2).ravel()
  TPR =tp / (tp + fn)
  TNR = tn / (tn + fp)

  #accuracy_PolyPhen2[fold]=((tp+tn)/(tp +tn+ fp+ fn))*100
  sensitivity_PolyPhen2[fold]=TPR*100
  specificity_PolyPhen2[fold]=TNR*100
  mcc_PolyPhen2[fold]= matthews_corrcoef(y_drgn, predictions_drgn_PolyPhen2)*100
  auc_PolyPhen2[fold] =round(auc, 3)*100

In [ ]:
ds=[sensitivity_D2D,  specificity_D2D,  mcc_D2D, auc_D2D] # list of dictionaries
test = pd.DataFrame(columns = ['metric', 'fold', 'value'])
fold_list, value_list = [],[]
for i, m in enumerate(ds):
  for key, value in m.items():
    fold_list.append(key)
    value_list.append(value)

test['metric'] = ['sensitivity']* len(sensitivity_D2D) + ['specificity']* len(sensitivity_D2D)+  ['mcc']* len(sensitivity_D2D)+  ['auc']* len(sensitivity_D2D)
test['fold'] = fold_list
test['value'] = value_list
test['predictor'] = ['D2D']* len(sensitivity_D2D) * len(ds)

ds=[sensitivity_DEOGEN2,  specificity_DEOGEN2,  mcc_DEOGEN2, auc_DEOGEN2] # list of dictionaries
df2 = pd.DataFrame(columns = ['metric', 'fold', 'value'])
fold_list, value_list = [],[]
for i, m in enumerate(ds):
  for key, value in m.items():
    fold_list.append(key)
    value_list.append(value)

df2['metric'] = ['sensitivity']* len(sensitivity_DEOGEN2) + ['specificity']* len(sensitivity_DEOGEN2)+  ['mcc']* len(sensitivity_DEOGEN2)+  ['auc']* len(sensitivity_DEOGEN2)
df2['fold'] = fold_list
df2['value'] = value_list
df2['predictor'] = ['DEOGEN2']* len(sensitivity_D2D) * len(ds)
test = test.append(df2, ignore_index=True)

ds=[sensitivity_EVE,  specificity_EVE,  mcc_EVE, auc_EVE] # list of dictionaries
df2 = pd.DataFrame(columns = ['metric', 'fold', 'value'])
fold_list, value_list = [],[]
for i, m in enumerate(ds):
  for key, value in m.items():
    fold_list.append(key)
    value_list.append(value)

df2['metric'] = ['sensitivity']* len(sensitivity_EVE) + ['specificity']* len(sensitivity_EVE)+  ['mcc']* len(sensitivity_EVE)+  ['auc']* len(sensitivity_EVE)
df2['fold'] = fold_list
df2['value'] = value_list
df2['predictor'] = ['EVE']* len(sensitivity_D2D) * len(ds)
test = test.append(df2, ignore_index=True)

ds=[sensitivity_FATHMM,  specificity_FATHMM,  mcc_FATHMM, auc_FATHMM] # list of dictionaries
df2 = pd.DataFrame(columns = ['metric', 'fold', 'value'])
fold_list, value_list = [],[]
for i, m in enumerate(ds):
  for key, value in m.items():
    fold_list.append(key)
    value_list.append(value)

df2['metric'] = ['sensitivity']* len(sensitivity_FATHMM) + ['specificity']* len(sensitivity_FATHMM)+  ['mcc']* len(sensitivity_FATHMM)+  ['auc']* len(sensitivity_FATHMM)
df2['fold'] = fold_list
df2['value'] = value_list
df2['predictor'] = ['FATHMM']* len(sensitivity_D2D) * len(ds)
test = test.append(df2, ignore_index=True)

ds=[sensitivity_SIFT,  specificity_SIFT,  mcc_SIFT, auc_SIFT] # list of dictionaries
df2 = pd.DataFrame(columns = ['metric', 'fold', 'value'])
fold_list, value_list = [],[]
for i, m in enumerate(ds):
  for key, value in m.items():
    fold_list.append(key)
    value_list.append(value)

df2['metric'] = ['sensitivity']* len(sensitivity_SIFT) + ['specificity']* len(sensitivity_SIFT)+  ['mcc']* len(sensitivity_SIFT)+  ['auc']* len(sensitivity_SIFT)
df2['fold'] = fold_list
df2['value'] = value_list
df2['predictor'] = ['SIFT']* len(sensitivity_D2D) * len(ds)
test = test.append(df2, ignore_index=True)

ds=[sensitivity_PolyPhen2,  specificity_PolyPhen2,  mcc_PolyPhen2, auc_PolyPhen2] # list of dictionaries
df2 = pd.DataFrame(columns = ['metric', 'fold', 'value'])
fold_list, value_list = [],[]
for i, m in enumerate(ds):
  for key, value in m.items():
    fold_list.append(key)
    value_list.append(value)

df2['metric'] = ['sensitivity']* len(sensitivity_PolyPhen2) + ['specificity']* len(sensitivity_PolyPhen2)+  ['mcc']* len(sensitivity_PolyPhen2)+  ['auc']* len(sensitivity_PolyPhen2)
df2['fold'] = fold_list
df2['value'] = value_list
df2['predictor'] = ['PolyPhen2']* len(sensitivity_D2D) * len(ds)
test = test.append(df2, ignore_index=True)

#fig = px.violin(test, y="metric", x="value", color="metric", box=True, points="all", width=800, height=400, title="5 folds D2D")
#fig.update_xaxes(range=[-60, 100])
#fig.show()
test_notmcc = test[test['metric'] != 'mcc']

In [ ]:
fig = px.box(test_notmcc, x="metric", y="value", color="predictor", color_discrete_map = {'D2D': '#6DA4AF', 'EVE': '#F9E393', 'FATHMM': '#9BC9E2','PolyPhen2': '#9DAF79','DEOGEN2': '#907F71', 'SIFT':'#635C92'}, width=1600, height=500)
#fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.update_layout({
'plot_bgcolor': 'rgba(255, 255, 255, 255)' ,
'paper_bgcolor': 'rgba(255, 255, 255, 255)',
})

fig.show()

In [ ]:
fig = px.box(test_notmcc, x="metric", y="value", color="predictor", width=1600, height=500)
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()

In [ ]:
test_mcc = test[test['metric'] == 'mcc']
fig = px.box(test_mcc, x="metric", y="value", color="predictor", color_discrete_map = {'D2D': '#6DA4AF', 'EVE': '#F9E393', 'FATHMM': '#9BC9E2','PolyPhen2': '#9DAF79','DEOGEN2': '#907F71', 'SIFT':'#635C92'}, width=600, height=500) #2292a7 #ffc907 #56B4E9 #82ad27 #695647 #4234a4
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.update_layout({
'plot_bgcolor': 'rgba(255, 255, 255, 255)' ,
'paper_bgcolor': 'rgba(255, 255, 255, 255)',
})
fig.show()

In [ ]:
test_mcc = test[test['metric'] == 'mcc']
fig = px.box(test_mcc, x="metric", y="value", color="predictor", width=600, height=500)
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()